In [1]:
import csv
import re
from io import BytesIO
import requests
import pdfplumber
import os
import pikepdf
import pandas as pd
from bs4 import BeautifulSoup
from PyPDF2 import PdfReader, PdfWriter
from pathlib import Path
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from folium import Choropleth
from folium.plugins import HeatMap
from geopy.geocoders import Nominatim
from fuzzywuzzy import process
import geopandas as gpd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import StandardScaler

## 0. Leitura dos Ficheiros

In [2]:
# Guardar todos os outputs numa única pasta
# output_folder = Path.cwd() / "output"
# output_folder.mkdir(parents=True, exist_ok=True)

# Exemplo: mapa.save(output_folder / 'mapa_de_calor_portugal_Danielle.html')

In [3]:
p = Path.cwd()
data_folder = p.parent.parent / "data"
shape_folder = p.parent.parent / "shape"

## 1. Incêndios

In [4]:
file_path = data_folder / "Dados Incêndios Urbanos por Freguesia 2014-2021 ISCTE.xlsx"
Fogo = pd.read_excel(file_path)
Fogo

,Ano,Mês,Distrito,CIM,DICO,Concelho,DICOFRE,Freguesia,CodNatureza,Natureza,Ocorrências,MeiosTerrestres,Duração Minutos
0,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10119,Valongo do Vouga,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,1.0,61.0
1,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10121,Águeda e Borralha,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,1.0,27.0
2,2014,1.0,Aveiro,Região de Aveiro,102.0,Albergaria-a-Velha,10209,Albergaria-a-Velha e Valmaior,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,4.0,259.0
3,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10312,Vila Nova de Monsarros,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,1.0,48.0
4,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10313,Vilarinho do Bairro,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,2.0,147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44439,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182321,Ranhados,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,3.0,110.0
44440,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182338,Faíl e Vila Chã de Sá,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,2.0,68.0
44441,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182341,Viseu,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,3.0,7.0,207.0
44442,2021,12.0,Viseu,Viseu Dão Lafões,1824.0,Vouzela,182414,Fataunços e Figueiredo Das Donas,2101.0,Riscos Tecnológicos / Incêndios Urbanos ou em ...,1.0,2.0,140.0


In [5]:
Fogo["Natureza"] = Fogo["Natureza"].str.split('/').str[-1].str.strip()
Fogo.dropna(subset=["Concelho"], inplace=True)
Fogo

,Ano,Mês,Distrito,CIM,DICO,Concelho,DICOFRE,Freguesia,CodNatureza,Natureza,Ocorrências,MeiosTerrestres,Duração Minutos
0,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10119,Valongo do Vouga,2101.0,Habitacional,1.0,1.0,61.0
1,2014,1.0,Aveiro,Região de Aveiro,101.0,Águeda,10121,Águeda e Borralha,2101.0,Habitacional,1.0,1.0,27.0
2,2014,1.0,Aveiro,Região de Aveiro,102.0,Albergaria-a-Velha,10209,Albergaria-a-Velha e Valmaior,2101.0,Habitacional,1.0,4.0,259.0
3,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10312,Vila Nova de Monsarros,2101.0,Habitacional,1.0,1.0,48.0
4,2014,1.0,Aveiro,Região de Aveiro,103.0,Anadia,10313,Vilarinho do Bairro,2101.0,Habitacional,1.0,2.0,147.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
44438,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182301,Abraveses,2101.0,Habitacional,3.0,7.0,171.0
44439,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182321,Ranhados,2101.0,Habitacional,1.0,3.0,110.0
44440,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182338,Faíl e Vila Chã de Sá,2101.0,Habitacional,1.0,2.0,68.0
44441,2021,12.0,Viseu,Viseu Dão Lafões,1823.0,Viseu,182341,Viseu,2101.0,Habitacional,3.0,7.0,207.0


In [6]:
pivot_table = Fogo.pivot_table(Fogo, 
                             index=['Ano', 'Mês', 'Distrito', 'Concelho'],
                             columns='Natureza',               
                             aggfunc='size',                   
                             fill_value=0)                     

pivot_table

Natureza                               Bibliotecas e Arquivos  \
Ano  Mês  Distrito Concelho                                     
2014 1.0  Aveiro   Albergaria-a-Velha                       0   
                   Anadia                                   0   
                   Aveiro                                   0   
                   Castelo de Paiva                         0   
                   Espinho                                  0   
...                                                       ...   
2021 12.0 Évora    Mora                                     0   
                   Vendas Novas                             0   
                   Viana do Alentejo                        0   
                   Vila Viçosa                              0   
                   Évora                                    0   

Natureza                               Desporto e Lazer  \
Ano  Mês  Distrito Concelho                               
2014 1.0  Aveiro   Albergaria-a-Velha                 0   
                   Anadia                             0   
                   Aveiro                             0   
                   Castelo de Paiva                   0   
                   Espinho                            0   
...                                                 ...   
2021 12.0 Évora    Mora                               0   
                   Vendas Novas                       0   
                   Viana do Alentejo                  0   
                   Vila Viçosa                        0   
                   Évora                              0   

Natureza                               Edifícios degradados ou devolutos  \
Ano  Mês  Distrito Concelho                                                
2014 1.0  Aveiro   Albergaria-a-Velha                                  0   
                   Anadia                                              0   
                   Aveiro                                              0   
                   Castelo de Paiva                                    0   
                   Espinho                                             0   
...                                                                  ...   
2021 12.0 Évora    Mora                                                0   
                   Vendas Novas                                        1   
                   Viana do Alentejo                                   0   
                   Vila Viçosa                                         0   
                   Évora                                               0   

Natureza                               Espectáculos e Reuniões Públicas  \
Ano  Mês  Distrito Concelho                                               
2014 1.0  Aveiro   Albergaria-a-Velha                                 0   
                   Anadia                                             0   
                   Aveiro                                             0   
                   Castelo de Paiva                                   0   
                   Espinho                                            0   
...                                                                 ...   
2021 12.0 Évora    Mora                                               0   
                   Vendas Novas                                       0   
                   Viana do Alentejo                                  0   
                   Vila Viçosa                                        0   
                   Évora                                              0   

Natureza                               Estacionamento de superfície  \
Ano  Mês  Distrito Concelho                                           
2014 1.0  Aveiro   Albergaria-a-Velha                             0   
                   Anadia                                         0   
                   Aveiro                                         0   
                   Castelo de Paiva                               

In [7]:
FogoPosto = Fogo.groupby(["Ano", "Mês", "Distrito", "Concelho"])[["Ocorrências", "MeiosTerrestres", "Duração Minutos"]].sum()
FogoPosto

Ocorrências  MeiosTerrestres  \
Ano  Mês  Distrito Concelho                                           
2014 1.0  Aveiro   Albergaria-a-Velha          1.0              4.0   
                   Anadia                      4.0              5.0   
                   Aveiro                      9.0             12.0   
                   Castelo de Paiva            2.0              4.0   
                   Espinho                     2.0              8.0   
...                                            ...              ...   
2021 12.0 Évora    Mora                        1.0              1.0   
                   Vendas Novas                3.0              3.0   
                   Viana do Alentejo           1.0              4.0   
                   Vila Viçosa                 1.0              2.0   
                   Évora                       2.0              7.0   

                                       Duração Minutos  
Ano  Mês  Distrito Concelho                             
2014 1.0  Aveiro   Albergaria-a-Velha            259.0  
                   Anadia                        275.0  
                   Aveiro                        804.0  
                   Castelo de Paiva              199.0  
                   Espinho                       363.0  
...                                                ...  
2021 12.0 Évora    Mora                           26.0  
                   Vendas Novas                  140.0  
                   Viana do Alentejo             191.0  
                   Vila Viçosa                    92.0  
                   Évora                         327.0  

[16686 rows x 3 columns]

In [8]:
FogoFinal = pd.merge(pivot_table, FogoPosto, how='inner', on=['Ano', 'Mês', 'Concelho', "Distrito"])
FogoFinal.reset_index(inplace=True)
FogoFinal

,Ano,Mês,Concelho,Distrito,Bibliotecas e Arquivos,Desporto e Lazer,Edifícios degradados ou devolutos,Espectáculos e Reuniões Públicas,Estacionamento de superfície,Estacionamento em profundidade ou silo,...,Hotelaria e Restauração,"Indústria, Oficina e Armazém","Militar, Forças de Segurança e Forças de Socorro",Museus e Galerias de Arte,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos
0,2014,1.0,Albergaria-a-Velha,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,4.0,259.0
1,2014,1.0,Anadia,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.0,5.0,275.0
2,2014,1.0,Aveiro,Aveiro,0,0,0,0,0,0,...,1,0,0,0,0,0,0,9.0,12.0,804.0
3,2014,1.0,Castelo de Paiva,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,4.0,199.0
4,2014,1.0,Espinho,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,8.0,363.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16681,2021,12.0,Mora,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,1.0,26.0
16682,2021,12.0,Vendas Novas,Évora,0,0,1,0,0,0,...,0,0,0,0,0,0,0,3.0,3.0,140.0
16683,2021,12.0,Viana do Alentejo,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,4.0,191.0
16684,2021,12.0,Vila Viçosa,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,2.0,92.0


## 2. Densidade Populacional

In [9]:
file_path = data_folder / "PORDATA_Densidade-populacional.xlsx"
Denso = pd.read_excel(file_path, header=11, usecols=range(17))
Denso.drop("Âmbito Geográfico", axis=1, inplace=True)
Denso = Denso.dropna()
Denso.drop(334, inplace=True)
Denso.rename(columns={'Anos': "Concelho"}, inplace=True)
Denso["Concelho"] = Denso["Concelho"].str.title()
Denso

,Concelho,2001,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos De Valdevez,55.2,52,51.5,50.6,49.7,49,48.4,47.9,47.4,47.1,46.8,46.6,46.5,46.4,46.2
1,Caminha,124.1,123.4,122.9,122.5,121.9,120.9,119.7,118.5,117.5,116.8,116.3,116.1,116.2,116.6,117.4
2,Melgaço,41.9,39.4,39,38.3,37.5,36.9,36.3,35.8,35.3,34.7,34.1,33.7,33.2,32.5,32.1
3,Monção,94.3,92,91.5,90.8,89.9,88.9,87.8,86.9,86.1,85.5,85,84.7,84.8,84.7,84.6
4,Paredes De Coura,69.1,67.2,66.9,66.1,65.3,64.8,64.2,63.8,63.4,63,62.8,62.8,63,63,62.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,193,204.3,204.6,204.6,203.5,201.2,198.1,195,192.9,192,192.3,193.7,194.8,195.3,196.2
304,Santa Cruz,448.9,505,520.4,528.3,527.2,523.7,519.1,514.5,511.5,510.4,510.7,512.3,518,524.7,531.2
305,Santana,91.2,83.1,81.6,80.1,78.5,77,75.4,73.8,72.4,71.2,70.4,69.9,69.4,68.6,68
306,São Vicente,76.5,74.1,73.2,71.7,70,68.5,66.6,65,63.8,62.8,62.1,61.7,61.8,61.8,61.7


## 3. População Residente

In [10]:
file_path = data_folder / "PORDATA_NUTS2013_Total.xlsx"
Viver = pd.read_excel(file_path, header=11, usecols=range(25))
Viver.drop("Âmbito Geográfico", axis=1, inplace=True)
Viver = Viver.dropna()
Viver.drop(333, inplace=True)
Viver.rename(columns={'Anos': "Concelho"}, inplace=True)
Viver["Concelho"] = Denso["Concelho"].str.title()
Viver

,Concelho,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos De Valdevez,0,24686,24603,24448,24259,24062,23854,23692,23519,...,21944,21664,21420,21227,21076,20926,20844,20826,20754,20693
1,Caminha,0,17055,17100,17088,17051,17013,16986,16959,16892,...,16500,16345,16177,16045,15940,15873,15848,15863,15917,16033
2,Melgaço,0,9971,9949,9889,9796,9715,9630,9538,9465,...,8785,8652,8531,8418,8279,8136,8029,7901,7755,7648
3,Monção,0,19925,19950,19919,19843,19773,19698,19615,19536,...,18782,18556,18361,18199,18063,17953,17890,17909,17898,17868
4,Paredes De Coura,0,9565,9556,9509,9459,9429,9404,9368,9321,...,8952,8869,8811,8761,8706,8677,8685,8711,8703,8676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,0,12523,12683,12827,12971,13087,13165,13247,13316,...,13158,12969,12765,12625,12570,12585,12679,12749,12788,12841
304,Santa Cruz,0,30301,31734,33120,34466,35805,37154,38512,39847,...,42685,42307,41932,41690,41602,41625,41753,42115,42666,43190
305,Santana,0,8750,8685,8605,8511,8395,8276,8169,8062,...,7357,7204,7049,6911,6800,6722,6677,6624,6554,6490
306,São Vicente,0,6179,6174,6138,6078,6020,5979,5946,5904,...,5398,5255,5124,5027,4949,4897,4867,4872,4875,4861


## 4. Setor de Atividade

### 4.1. Primário

In [11]:
file_path = data_folder / "SetorDeAtividade.xlsx"
WorkP = pd.read_excel(file_path, header=11, usecols=[0,1,6,7])
WorkP.drop("Âmbito Geográfico", axis=1, inplace=True)
WorkP = WorkP.dropna()
WorkP.drop(333, inplace=True)
WorkP.rename(columns={'Anos': "Concelho"}, inplace=True)
WorkP["Concelho"] = WorkP["Concelho"].str.title()
WorkP

,Concelho,2011,2021
0,Arcos De Valdevez,514,374
1,Caminha,316,213
2,Melgaço,258,193
3,Monção,377,432
4,Paredes De Coura,185,151
...,...,...,...
303,Ribeira Brava,156,119
304,Santa Cruz,439,318
305,Santana,410,229
306,São Vicente,100,105


### 4.2. Secundário

In [12]:
file_path = data_folder / "SetorDeAtividade.xlsx"
WorkS = pd.read_excel(file_path, header=11, usecols=[0,1,12,13])
WorkS.drop("Âmbito Geográfico", axis=1, inplace=True)
WorkS = WorkS.dropna()
WorkS.drop(333, inplace=True)
WorkS.rename(columns={'2011.1': 2011}, inplace=True)
WorkS.rename(columns={'2021.1': 2021}, inplace=True)
WorkS.rename(columns={'Anos': "Concelho"}, inplace=True)
WorkS["Concelho"] = WorkS["Concelho"].str.title()
WorkS

,Concelho,2011,2021
0,Arcos De Valdevez,2369,2486
1,Caminha,1561,1634
2,Melgaço,595,579
3,Monção,2062,2058
4,Paredes De Coura,1253,1513
...,...,...,...
303,Ribeira Brava,1327,994
304,Santa Cruz,2777,2260
305,Santana,438,348
306,São Vicente,449,293


### 4.3. Setor Terciário

In [13]:
file_path = data_folder / "SetorDeAtividade.xlsx"
WorkT = pd.read_excel(file_path, header=11, usecols=[0,1,18,19])
WorkT.drop("Âmbito Geográfico", axis=1, inplace=True)
WorkT = WorkT.dropna()
WorkT.drop(333, inplace=True)
WorkT.rename(columns={'2011.2': 2011}, inplace=True)
WorkT.rename(columns={'2021.2': 2021}, inplace=True)
WorkT.rename(columns={'Anos': "Concelho"}, inplace=True)
WorkT["Concelho"] = WorkT["Concelho"].str.title()
WorkT

,Concelho,2011,2021
0,Arcos De Valdevez,4175,4141
1,Caminha,4386,4438
2,Melgaço,1767,1657
3,Monção,4077,4182
4,Paredes De Coura,1910,1750
...,...,...,...
303,Ribeira Brava,3454,3441
304,Santa Cruz,16902,16317
305,Santana,1855,1722
306,São Vicente,1351,1281


## 5. Indíce de Envelhecimento

In [14]:
file_path = data_folder / "IndiceEnvelhecimento.xlsx"
Velho = pd.read_excel(file_path, header=11, usecols=range(25))
Velho.drop("Âmbito Geográfico", axis=1, inplace=True)
Velho.dropna(inplace=True)
Velho.drop(333, inplace=True)
Velho.rename(columns={'Anos': "Concelho"}, inplace=True)
Velho["Concelho"] = Velho["Concelho"].str.title()
Velho

,Concelho,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos De Valdevez,0,208.1,216.4,223.7,231.3,238.7,244,248.3,253.1,...,273.1,282.8,296,310.1,324.8,339.6,355.1,369.2,379.3,391.6
1,Caminha,0,138.1,141.1,145,147.3,149.9,153.1,156.4,161.3,...,203.4,209.8,218.9,229.2,239.7,247.6,253.3,258.9,265.8,272.3
2,Melgaço,0,297.4,305.3,311.1,321.1,332.3,348.2,367.2,376.7,...,393.4,410.7,433.9,451.1,473.6,501,526.9,548.2,563.3,577
3,Monção,0,209.7,217.6,222.3,227.2,234.8,242,246.3,246.3,...,277.2,284.3,292.3,301,310.2,318.2,324.1,332.2,343.5,351.5
4,Paredes De Coura,0,204.6,208.8,211.6,214.2,216.5,218.1,216.2,213.4,...,219,226,232.5,237.7,244.3,250.3,256.3,267.3,278.4,289.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,0,80.9,81,78.5,76.5,76.1,76,76,76.9,...,96.4,102.2,108,114.8,122.5,131.4,141.7,152,162.1,169.5
304,Santa Cruz,0,56.1,55.4,54.4,53.1,52.1,51.2,50.4,49.7,...,57.9,60.8,63.9,67.8,72.7,78.1,83.8,89.9,95.7,101.1
305,Santana,0,128.1,136.8,143.8,149,151.7,153.7,159.1,165.3,...,189.7,201.1,211.2,222.4,234.8,242.8,253,265.4,279.7,290.4
306,São Vicente,0,121.5,127,131.1,133.7,136.7,141.1,146,152,...,169.4,180.6,190.5,197.8,206,215.3,228.4,243.7,258.3,273.9


## 6. Zonas Rurais

In [15]:
file_path = data_folder / "ZonasCount.csv"
ZonasRurais = pd.read_csv(file_path)
ZonasRurais["Rural"] = ZonasRurais["Zona Rural"] / (ZonasRurais["Zona Urbana"] + ZonasRurais["Zona Rural"])
ZonasRurais

,Distrito,Concelho,Zona Rural,Zona Urbana,Rural
0,Aveiro,Albergaria-A-Velha,6,0,1.00
1,Aveiro,Anadia,10,0,1.00
2,Aveiro,Arouca,16,0,1.00
3,Aveiro,Aveiro,0,10,0.00
4,Aveiro,Castelo De Paiva,6,0,1.00
...,...,...,...,...,...
273,Évora,Reguengos De Monsaraz,4,0,1.00
274,Évora,Vendas Novas,2,0,1.00
275,Évora,Viana Do Alentejo,3,0,1.00
276,Évora,Vila Viçosa,4,0,1.00


## 7. IPMA

In [16]:
# Caminho do arquivo Excel
file_path = data_folder / "Dados_diarios_T_Vento_Prec_capitaisDistrito_2013_a_2023.xlsx"

# Lista de nomes dos distritos
distritos = ['Viana do Castelo', 'Braga', 'Vila Real', 'Bragança', 'Porto', 'Aveiro', 'Viseu', 'Guarda',
             'Coimbra', 'Castelo Branco', 'Leiria', 'Santarém', 'Portalegre', 'Lisboa', 'Setúbal', 'Évora',
             'Beja', 'Faro']

# Lista completa de colunas a serem lidas
colunasTempo = ['Ano', 'Mês', 'Dia'] + distritos

colunas_indices = list(range(len(colunasTempo)))

# Leitura do arquivo Excel para cada tabela
T_MED = pd.read_excel(file_path, sheet_name=1, names=colunasTempo)
T_MAX = pd.read_excel(file_path, sheet_name=2, names=colunasTempo)
T_MIN = pd.read_excel(file_path, sheet_name=3, names=colunasTempo)
DD_MED = pd.read_excel(file_path, sheet_name=4, names=colunasTempo)
DD_FFX = pd.read_excel(file_path, sheet_name=5, names=colunasTempo)
FF_MED = pd.read_excel(file_path, sheet_name=6, names=colunasTempo)
FF_MAX = pd.read_excel(file_path, sheet_name=7, names=colunasTempo)
PR_QTD = pd.read_excel(file_path, sheet_name=8, names=colunasTempo)

tabelas = [T_MED, T_MAX, T_MIN, DD_MED, DD_FFX, FF_MED, FF_MAX, PR_QTD]
for i in tabelas:
    i.rename(columns={'Viana do Castelo': "Viana Do Castelo"}, inplace=True)

T_MED

,Ano,Mês,Dia,Viana Do Castelo,Braga,Vila Real,Bragança,Porto,Aveiro,Viseu,...,Coimbra,Castelo Branco,Leiria,Santarém,Portalegre,Lisboa,Setúbal,Évora,Beja,Faro
0,2013,1,1,9.6,9.0,8.2,7.2,10.5,10.5,7.3,...,10.3,8.0,9.9,10.4,7.9,12.3,11.9,9.4,10.2,13.6
1,2013,1,2,7.9,7.1,6.6,4.8,9.0,10.0,7.3,...,9.0,9.2,7.3,10.4,9.0,11.8,11.7,8.8,9.0,12.7
2,2013,1,3,8.6,9.2,7.6,5.9,11.5,13.4,8.3,...,11.3,9.0,7.6,10.0,9.4,10.9,9.7,7.6,8.9,13.0
3,2013,1,4,8.0,7.3,7.0,5.4,11.1,11.0,9.4,...,11.2,7.8,7.7,9.4,9.3,9.9,7.3,7.6,8.7,13.2
4,2013,1,5,6.6,5.8,6.2,5.2,9.2,9.3,9.7,...,9.5,8.3,7.8,9.4,10.2,10.5,8.0,7.9,9.8,13.3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4012,2023,12,27,13.3,-990.0,2.2,0.4,11.5,-990.0,7.3,...,8.2,2.7,7.7,7.6,5.6,7.7,8.0,5.0,6.2,10.9
4013,2023,12,28,12.7,-990.0,6.9,4.6,13.6,-990.0,8.8,...,12.0,6.1,10.9,9.5,9.7,9.8,10.2,7.8,9.2,13.4
4014,2023,12,29,12.0,-990.0,9.2,7.0,12.2,-990.0,9.3,...,12.1,7.4,12.1,9.8,9.5,10.0,9.7,8.9,10.2,14.5
4015,2023,12,30,12.6,-990.0,9.7,7.8,12.6,-990.0,9.0,...,11.2,8.4,11.3,11.0,10.0,10.9,9.9,8.2,10.8,13.9


## 8. Eventos Excecionais

In [17]:
file_path = data_folder / "EventosLimpos.csv"
Quebras = pd.read_csv(file_path)
Quebras.rename(columns={'Concelho origem do incidente': "Concelho"}, inplace=True)
Quebras.reset_index(drop = True, inplace=True)
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),END (MWh),SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563","0,2135252","0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009","0,2134525","0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047","1,3991121","0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000","0,0000000","0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377","0,3217694","0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,0,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017
7729,EDPD_2017_NOV_T_63,Santa Maria da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,0,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017
7730,EDPD_2017_NOV_T_64,Santa Maria da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,0,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017
7731,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,0,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017


In [18]:
# file_path = data_folder / "eventos.parquet"
# Quebras = pd.read_parquet(file_path)
# Quebras.dropna(inplace=True)
# Quebras.reset_index(drop = True, inplace=True)
# Quebras['Ano'] = Quebras['data_do_incidente'].dt.year
# Quebras['Mês'] = Quebras['data_do_incidente'].dt.month
# Quebras['Dia'] = Quebras['data_do_incidente'].dt.day
# Quebras.rename(columns={'concelho_origem_do_incidente': "Concelho"}, inplace=True)
# Quebras["Concelho"] = Quebras["Concelho"].str.title()
# Quebras['aprovado'] = Quebras['aprovado'].astype(int)
# mapeamento = {'BT': 1, 'MT': 2, 'AT': 3}
# Quebras['nt'] = Quebras['nt'].map(mapeamento)
# Quebras["Concelho"] = Quebras["Concelho"].replace({'Ponte de Sor': 'Ponte De Sôr'})
# Quebras

## 9. Junção dos dados

### 9.1. Distritos

In [19]:
file_path = shape_folder / "concelhos.shp"
Concelho = gpd.read_file(file_path)
Concelho
Concelho.rename(columns={'NAME_1': 'Distrito'}, inplace=True)
Concelho.rename(columns={'NAME_2': 'Concelho'}, inplace=True)
Concelho['Concelho'] = Concelho['Concelho'].str.title()
Concelho['Distrito'] = Concelho['Distrito'].str.title()
Concelho = Concelho[Concelho['Distrito'] != 'Azores']

# Crie um dicionário vazio para armazenar os concelhos de cada distrito
dicionario_concelhos = {}

# Itere sobre os distritos únicos na coluna 'Distrito'
for distrito in Concelho['Distrito'].unique():
    # Filtrar os concelhos para o distrito atual
    concelhos_distrito = Concelho.loc[Concelho['Distrito'] == distrito, 'Concelho'].tolist()
    # Adicionar os concelhos ao dicionário
    dicionario_concelhos[distrito] = concelhos_distrito
dicionario_concelhos['Braga'].append('Guimarães')
dicionario_concelhos['Portalegre'].append('Ponte De Sor')
dicionario_concelhos['Faro'].append('Tavira')

In [20]:
# Substituir "da" por "Da" na coluna especificada
Quebras['Concelho'] = Quebras['Concelho'].str.replace(' da ', ' Da ')
Quebras['Concelho'] = Quebras['Concelho'].str.replace(' de ', ' De ')
Quebras['Concelho'] = Quebras['Concelho'].str.replace(' do ', ' Do ')
Quebras['Concelho'] = Quebras['Concelho'].str.replace(' dos ', ' Dos ')
Quebras['Concelho'] = Quebras['Concelho'].str.replace('-a-', '-A-')
Quebras['Concelho'] = Quebras['Concelho'].str.replace('-o-', '-O-')
Quebras['Concelho'] = Quebras['Concelho'].str.replace(' à ', ' À ')

In [21]:
def mapear_distrito(concelho):
    for distrito, concelhos in dicionario_concelhos.items():
        if concelho in concelhos:
            return distrito
    return None  # Retorna None se o concelho não for encontrado no dicionário

# Aplicar a função mapear_distrito à coluna 'Concelhos' e criar uma nova coluna 'Distrito' no DataFrame Quebras
Quebras['Distrito'] = Quebras['Concelho'].apply(mapear_distrito)
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano,Distrito
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,"0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018,Setúbal
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,"0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018,Castelo Branco
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,"0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018,Porto
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,"0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018,Lisboa
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,"0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018,Portalegre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017,Aveiro
7729,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Aveiro
7730,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017,Aveiro
7731,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Santarém


#### 9.1.1 TODO - Não sei o que fazer com estes

In [22]:
Quebras[Quebras["Distrito"].isna()]["Concelho"].unique()

array(['Território De Portugal Continental', 'Portugal Continental',
       'Região Centro De Portugal',
       '(Distritos) Aveiro, Castelo Branco, Portalegre, Porto, Santarém e Viseu',
       'Distritos De Beja e Faro',
       'Distritos De Aveiro, Braga, Bragança, Castelo Branco, Coimbra, Guarda, Porto, Vila Real e Viseu',
       'Distritos De Aveiro, Braga, Castelo Branco, Coimbra, Évora, Faro, Guarda, Leiria, Portalegre, Porto, Viana Do Castelo, Vila Real e Viseu',
       'Tavira, Silves, Loulé, Faro, Serpa, Lagoa, Beja', 'Região Norte',
       'Região Aveiro - Feira', 'Região Leiria - Caldas'], dtype=object)

In [23]:
Concelho[Concelho['Concelho'].str.contains("Vinhos")]

,ID_0,ISO,NAME_0,ID_1,Distrito,ID_2,Concelho,HASC_2,CCN_2,CCA_2,TYPE_2,ENGTYPE_2,NL_NAME_2,VARNAME_2,geometry
158,182,PRT,Portugal,11,Leiria,157,Figueiró Dos Vinhos,PT.LE.FV,0,1008,Concelho,Municipality,None,None,"POLYGON ((-8.23894 40.06718, -8.23880 40.06716..."
168,182,PRT,Portugal,12,Lisboa,167,Arruda Dos Vinhos,PT.LI.AV,0,1102,Concelho,Municipality,None,None,"POLYGON ((-9.09276 39.02848, -9.09268 39.02848..."


In [24]:
Quebras = Quebras.dropna(subset=['Distrito'])

In [25]:
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano,Distrito
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,"0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018,Setúbal
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,"0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018,Castelo Branco
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,"0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018,Porto
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,"0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018,Lisboa
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,"0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018,Portalegre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017,Aveiro
7729,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Aveiro
7730,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017,Aveiro
7731,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Santarém


In [26]:
Quebras["Causa do incidente"].unique()

array(['Escavações', 'Abate De Árvores', 'Deslizamento De Terras',
       'Veículos', 'Aves', 'Animais Não Aves', 'Incêndio',
       'Malfeitoria/Vandalismo', 'Terramoto',
       'Neve/Gelo/Granizo Condição Excecional',
       'Corpos Estranhos Na Rede', 'Vento Intensidade Excepcional',
       'Descarga Atmosférica Directa', 'Intervenções À Superficie',
       'Inundações Imprevisíveis', 'Furtos',
       'Risco Iminente Pessoas Ou Bens', 'Sabotagem',
       'Desabamento De Muros/Outras Cons',
       'Desabamento De Muros/Outras Construções',
       'Trabalhos De Terceiros ( Gruas,... )',
       'Ruptura De Canal. De Fluídos', 'Malfeitorioa (Vandalismo)',
       'Rnt - Caso Fortuito/Força Maior/Razão Segurança'], dtype=object)

In [27]:
for i in Quebras["Causa do incidente"].unique():
        if "Malfeitorioa" in i:
            linha = Quebras[Quebras["Causa do incidente"] == i].index
            Quebras.loc[linha, "Causa do incidente"] = "Malfeitoria/Vandalismo"

        elif "Desabamento De Muros/Outras Cons" in i:
            linha = Quebras[Quebras["Causa do incidente"] == i].index
            Quebras.loc[linha, "Causa do incidente"] = "Desabamento De Muros/Outras Construções"
Quebras["Causa do incidente"].unique()

array(['Escavações', 'Abate De Árvores', 'Deslizamento De Terras',
       'Veículos', 'Aves', 'Animais Não Aves', 'Incêndio',
       'Malfeitoria/Vandalismo', 'Terramoto',
       'Neve/Gelo/Granizo Condição Excecional',
       'Corpos Estranhos Na Rede', 'Vento Intensidade Excepcional',
       'Descarga Atmosférica Directa', 'Intervenções À Superficie',
       'Inundações Imprevisíveis', 'Furtos',
       'Risco Iminente Pessoas Ou Bens', 'Sabotagem',
       'Desabamento De Muros/Outras Construções',
       'Trabalhos De Terceiros ( Gruas,... )',
       'Ruptura De Canal. De Fluídos',
       'Rnt - Caso Fortuito/Força Maior/Razão Segurança'], dtype=object)

## 9.2. Incêndios - TODO

In [28]:
FogoFinal

,Ano,Mês,Concelho,Distrito,Bibliotecas e Arquivos,Desporto e Lazer,Edifícios degradados ou devolutos,Espectáculos e Reuniões Públicas,Estacionamento de superfície,Estacionamento em profundidade ou silo,...,Hotelaria e Restauração,"Indústria, Oficina e Armazém","Militar, Forças de Segurança e Forças de Socorro",Museus e Galerias de Arte,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos
0,2014,1.0,Albergaria-a-Velha,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,4.0,259.0
1,2014,1.0,Anadia,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.0,5.0,275.0
2,2014,1.0,Aveiro,Aveiro,0,0,0,0,0,0,...,1,0,0,0,0,0,0,9.0,12.0,804.0
3,2014,1.0,Castelo de Paiva,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,4.0,199.0
4,2014,1.0,Espinho,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,8.0,363.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16681,2021,12.0,Mora,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,1.0,26.0
16682,2021,12.0,Vendas Novas,Évora,0,0,1,0,0,0,...,0,0,0,0,0,0,0,3.0,3.0,140.0
16683,2021,12.0,Viana do Alentejo,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,4.0,191.0
16684,2021,12.0,Vila Viçosa,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,2.0,92.0


In [29]:
# Obter valores únicos da coluna "Distrito" do DataFrame "Quebras"
valores_Distrito = Quebras['Distrito'].unique()

# Iterar sobre os valores únicos da coluna "Distrito" do DataFrame "FogoFinal"
for valor in FogoFinal['Distrito'].unique():
    # Verificar se o valor está presente na coluna "Distrito" do DataFrame "Distrito"
    if valor not in valores_Distrito:
        # Encontrar o nome mais semelhante na coluna "Distrito" do DataFrame "Distrito"
        correspondencia, pontuacao = process.extractOne(valor, valores_Distrito)
        
        # Substituir o valor incorreto pelo nome correspondente
        FogoFinal['Distrito'].replace({valor: correspondencia}, inplace=True)

# Exibir o DataFrame "FogoFinal" após a correção dos nomes incorretos
FogoFinal

,Ano,Mês,Concelho,Distrito,Bibliotecas e Arquivos,Desporto e Lazer,Edifícios degradados ou devolutos,Espectáculos e Reuniões Públicas,Estacionamento de superfície,Estacionamento em profundidade ou silo,...,Hotelaria e Restauração,"Indústria, Oficina e Armazém","Militar, Forças de Segurança e Forças de Socorro",Museus e Galerias de Arte,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos
0,2014,1.0,Albergaria-a-Velha,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,4.0,259.0
1,2014,1.0,Anadia,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.0,5.0,275.0
2,2014,1.0,Aveiro,Aveiro,0,0,0,0,0,0,...,1,0,0,0,0,0,0,9.0,12.0,804.0
3,2014,1.0,Castelo de Paiva,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,4.0,199.0
4,2014,1.0,Espinho,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,8.0,363.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16681,2021,12.0,Mora,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,1.0,26.0
16682,2021,12.0,Vendas Novas,Évora,0,0,1,0,0,0,...,0,0,0,0,0,0,0,3.0,3.0,140.0
16683,2021,12.0,Viana do Alentejo,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,4.0,191.0
16684,2021,12.0,Vila Viçosa,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,2.0,92.0


In [30]:
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,SAIFI BT (#),SAIDI BT (min),Qualidade de Energia Elétrica,Qualidade de Serviço Comercial,Fundamentação,Decisão,Dia,Mês,Ano,Distrito
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,"0,0001357","0,0075338",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,2,1,2018,Setúbal
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,"0,0000900","0,0084007",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,2,1,2018,Castelo Branco
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,"0,0003141","0,0574566",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,4,1,2018,Porto
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,"0,0000167","0,0029788",n.a.,n.a.,Não reúne condições para aprovação: - Após par...,Não Aprovado,5,1,2018,Lisboa
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,"0,0000518","0,0023477",n.a.,n.a.,Reúne condições para aprovação: - Reúne os req...,Aprovado,9,1,2018,Portalegre
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7728,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,"0,000000","0,000072",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,29,11,2017,Aveiro
7729,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,"0,000000","0,000015",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Aveiro
7730,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,"0,000000","0,000040",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,23,11,2017,Aveiro
7731,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,"0,000028","0,004271",n.a.,n.a.,Não reúne condições para aprovação: - Não cump...,Não Aprovado,3,11,2017,Santarém


In [31]:
# Obter valores únicos da coluna "Concelho" do DataFrame "Quebras"
valores_concelho = Quebras[['Concelho', 'Distrito']].apply(lambda x: (x['Concelho'], x['Distrito']), axis=1).tolist()

# Iterar sobre os valores únicos da coluna "Concelho" do DataFrame "FogoFinal"
for concelho, distrito in FogoFinal[['Concelho', 'Distrito']].apply(lambda x: (x['Concelho'], x['Distrito']), axis=1).unique():
    # Filtrar os concelhos pelo distrito correspondente
    concelhos_no_distrito = Quebras[Quebras['Distrito'] == distrito]['Concelho'].unique()
    
    # Verificar se o concelho está presente nos concelhos Distrito
    if concelho not in concelhos_no_distrito:
        # Encontrar o nome mais semelhante nos concelhos Distrito
        correspondencia, pontuacao = process.extractOne(concelho, concelhos_no_distrito)
        if pontuacao: 
            print(f"Correspondência de '{concelho}' para -> {correspondencia} (Pontuação: {pontuacao})")
            
            # Substituir o concelho incorreto pelo nome correspondente
            FogoFinal.loc[(FogoFinal['Concelho'] == concelho) & (FogoFinal['Distrito'] == distrito), 'Concelho'] = correspondencia

# Exibir o DataFrame "FogoFinal" após a correção dos nomes incorretos
FogoFinal

Correspondência de 'Albergaria-a-Velha' para -> Albergaria-A-Velha (Pontuação: 100)
Correspondência de 'Castelo de Paiva' para -> Castelo De Paiva (Pontuação: 100)
Correspondência de 'Oliveira de Azeméis' para -> Oliveira De Azeméis (Pontuação: 100)
Correspondência de 'Oliveira do Bairro' para -> Oliveira Do Bairro (Pontuação: 100)
Correspondência de 'Santa Maria da Feira' para -> Santa Maria Da Feira (Pontuação: 100)
Correspondência de 'Sever do Vouga' para -> Sever Do Vouga (Pontuação: 100)
Correspondência de 'São João da Madeira' para -> São João Da Madeira (Pontuação: 100)
Correspondência de 'Vale de Cambra' para -> Vale De Cambra (Pontuação: 100)
Correspondência de 'Cabeceiras de Basto' para -> Cabeceiras De Basto (Pontuação: 100)
Correspondência de 'Celorico de Basto' para -> Celorico De Basto (Pontuação: 100)
Correspondência de 'Póvoa de Lanhoso' para -> Póvoa De Lanhoso (Pontuação: 100)
Correspondência de 'Terras de Bouro' para -> Terras De Bouro (Pontuação: 100)
Correspondênci

,Ano,Mês,Concelho,Distrito,Bibliotecas e Arquivos,Desporto e Lazer,Edifícios degradados ou devolutos,Espectáculos e Reuniões Públicas,Estacionamento de superfície,Estacionamento em profundidade ou silo,...,Hotelaria e Restauração,"Indústria, Oficina e Armazém","Militar, Forças de Segurança e Forças de Socorro",Museus e Galerias de Arte,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos
0,2014,1.0,Albergaria-A-Velha,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,4.0,259.0
1,2014,1.0,Anadia,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,4.0,5.0,275.0
2,2014,1.0,Aveiro,Aveiro,0,0,0,0,0,0,...,1,0,0,0,0,0,0,9.0,12.0,804.0
3,2014,1.0,Castelo De Paiva,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,4.0,199.0
4,2014,1.0,Espinho,Aveiro,0,0,0,0,0,0,...,0,0,0,0,0,0,0,2.0,8.0,363.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16681,2021,12.0,Mora,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,1.0,26.0
16682,2021,12.0,Vendas Novas,Évora,0,0,1,0,0,0,...,0,0,0,0,0,0,0,3.0,3.0,140.0
16683,2021,12.0,Viana Do Alentejo,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,4.0,191.0
16684,2021,12.0,Vila Viçosa,Évora,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1.0,2.0,92.0


In [32]:
# Juntar a Base de dados dos incêndios
Quebras = pd.merge(Quebras, FogoFinal, on=['Ano', 'Mês', 'Concelho', 'Distrito'], how='left')
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Hotelaria e Restauração,"Indústria, Oficina e Armazém","Militar, Forças de Segurança e Forças de Socorro",Museus e Galerias de Arte,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,33.0,2443.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,439.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,18.0,1358.0
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,361.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,7.0,429.0
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,0.0,4.0,0.0,0.0,1.0,1.0,1.0,11.0,19.0,1002.0
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,0.0,4.0,0.0,0.0,1.0,1.0,1.0,11.0,19.0,1002.0
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,120.0


In [33]:
Quebras.fillna(0, inplace=True)
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Hotelaria e Restauração,"Indústria, Oficina e Armazém","Militar, Forças de Segurança e Forças de Socorro",Museus e Galerias de Arte,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,2.0,0.0,0.0,0.0,0.0,0.0,0.0,13.0,33.0,2443.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,439.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,6.0,18.0,1358.0
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,361.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,7.0,429.0
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,0.0,4.0,0.0,0.0,1.0,1.0,1.0,11.0,19.0,1002.0
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,0.0,4.0,0.0,0.0,1.0,1.0,1.0,11.0,19.0,1002.0
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,120.0


## 9.3. Densidade Populacional

In [34]:
# Lista de concelhos da base de dados 'Denso'
concelhos_denso = Denso['Concelho'].unique()

# Lista de concelhos da base de dados 'Quebras'
concelhos_quebras = Quebras['Concelho'].unique()

# Função para encontrar a correspondência fuzzy entre os concelhos
def encontrar_correspondencia(concelho_denso, concelhos_quebras):
    melhor_correspondencia, pontuacao = process.extractOne(concelho_denso, concelhos_quebras)
    return melhor_correspondencia, pontuacao

# Criar um dicionário de mapeamento de correspondências fuzzy
mapeamento_correspondencias = {}

# Iterar sobre os concelhos da base de dados 'Denso' e encontrar correspondências na base de dados 'Quebras'
for concelho_denso in concelhos_denso:
    # Encontrar a correspondência na base de dados 'Quebras'
    correspondencia_atual, pontuacao = encontrar_correspondencia(concelho_denso, concelhos_quebras)
    
    # Adicionar a correspondência ao dicionário de mapeamento
    mapeamento_correspondencias[concelho_denso] = (correspondencia_atual, pontuacao)

# Atualizar a base de dados 'Denso' com as correspondências encontradas
for concelho_denso, (correspondencia_atual, pontuacao) in mapeamento_correspondencias.items():
    if pontuacao >= 90:
        Denso.loc[Denso['Concelho'] == concelho_denso, 'Concelho'] = correspondencia_atual
    
        # Exibir a correspondência e a pontuação de similaridade
        print(f"Correspondência de '{concelho_denso}' para -> {correspondencia_atual} (Pontuação: {pontuacao})")

# Resultado final
Denso


Correspondência de 'Arcos De Valdevez' para -> Arcos De Valdevez (Pontuação: 100)
Correspondência de 'Caminha' para -> Caminha (Pontuação: 100)
Correspondência de 'Melgaço' para -> Melgaço (Pontuação: 100)
Correspondência de 'Monção' para -> Monção (Pontuação: 100)
Correspondência de 'Paredes De Coura' para -> Paredes De Coura (Pontuação: 100)
Correspondência de 'Ponte Da Barca' para -> Ponte Da Barca (Pontuação: 100)
Correspondência de 'Ponte De Lima' para -> Ponte De Lima (Pontuação: 100)
Correspondência de 'Valença' para -> Valença (Pontuação: 100)
Correspondência de 'Viana Do Castelo' para -> Viana Do Castelo (Pontuação: 100)
Correspondência de 'Vila Nova De Cerveira' para -> Vila Nova De Cerveira (Pontuação: 100)
Correspondência de 'Amares' para -> Amares (Pontuação: 100)
Correspondência de 'Barcelos' para -> Barcelos (Pontuação: 100)
Correspondência de 'Braga' para -> Braga (Pontuação: 100)
Correspondência de 'Esposende' para -> Esposende (Pontuação: 100)
Correspondência de 'Terr

,Concelho,2001,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos De Valdevez,55.2,52,51.5,50.6,49.7,49,48.4,47.9,47.4,47.1,46.8,46.6,46.5,46.4,46.2
1,Caminha,124.1,123.4,122.9,122.5,121.9,120.9,119.7,118.5,117.5,116.8,116.3,116.1,116.2,116.6,117.4
2,Melgaço,41.9,39.4,39,38.3,37.5,36.9,36.3,35.8,35.3,34.7,34.1,33.7,33.2,32.5,32.1
3,Monção,94.3,92,91.5,90.8,89.9,88.9,87.8,86.9,86.1,85.5,85,84.7,84.8,84.7,84.6
4,Paredes De Coura,69.1,67.2,66.9,66.1,65.3,64.8,64.2,63.8,63.4,63,62.8,62.8,63,63,62.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,193,204.3,204.6,204.6,203.5,201.2,198.1,195,192.9,192,192.3,193.7,194.8,195.3,196.2
304,Santa Cruz,448.9,505,520.4,528.3,527.2,523.7,519.1,514.5,511.5,510.4,510.7,512.3,518,524.7,531.2
305,Santana,91.2,83.1,81.6,80.1,78.5,77,75.4,73.8,72.4,71.2,70.4,69.9,69.4,68.6,68
306,São Vicente,76.5,74.1,73.2,71.7,70,68.5,66.6,65,63.8,62.8,62.1,61.7,61.8,61.8,61.7


In [35]:
for index, row in Quebras.iterrows():
    # Obtendo o ano e o concelho da linha atual
    ano = row['Ano']
    if ano == 2023:
        ano = 2022
    concelho = row['Concelho']
    
    # Verificando se o ano existe como coluna na tabela 'Denso'
    if ano in Denso.columns:
        # Verificando se o concelho está presente na tabela 'Denso'
        if concelho in Denso['Concelho'].values:
            # Filtrando a linha correspondente na tabela 'Denso' com base no concelho
            linha_denso = Denso[Denso['Concelho'] == concelho]
            
            # Obtendo o valor para o ano desejado
            valor = linha_denso[ano].values[0]
        else:
            # Atribuindo valor nulo se o concelho não estiver presente na tabela 'Denso'
            valor = None
    else:
        # Atribuindo valor nulo se o ano não estiver presente como coluna na tabela 'Denso'
        valor = None
    
    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'Densidade Populacional'] = valor

Quebras


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,"Indústria, Oficina e Armazém","Militar, Forças de Segurança e Forças de Socorro",Museus e Galerias de Arte,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos,Densidade Populacional
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,0.0,0.0,0.0,0.0,0.0,0.0,13.0,33.0,2443.0,2516.2
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.1
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,0.0,0.0,0.0,0.0,0.0,0.0,3.0,6.0,439.0,486.5
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,1.0,0.0,0.0,0.0,0.0,0.0,6.0,18.0,1358.0,198.6
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4.0,361.0,10.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,4.0,7.0,429.0,369.4
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,4.0,0.0,0.0,1.0,1.0,1.0,11.0,19.0,1002.0,631.8
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,4.0,0.0,0.0,1.0,1.0,1.0,11.0,19.0,1002.0,631.8
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,3.0,120.0,100.7


In [36]:
Quebras[Quebras["Densidade Populacional"].isna()]

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,"Indústria, Oficina e Armazém","Militar, Forças de Segurança e Forças de Socorro",Museus e Galerias de Arte,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos,Densidade Populacional


## 9.4. População Residente

In [37]:
# Lista de concelhos da base de dados 'Viver'
concelhos_viver = Viver['Concelho'].unique()

# Lista de concelhos da base de dados 'Quebras'
concelhos_quebras = Quebras['Concelho'].unique()

# Função para encontrar a correspondência fuzzy entre os concelhos
def encontrar_correspondencia(concelho_viver, concelhos_quebras):
    melhor_correspondencia, pontuacao = process.extractOne(concelho_viver, concelhos_quebras)
    return melhor_correspondencia, pontuacao

# Criar um dicionário de mapeamento de correspondências fuzzy
mapeamento_correspondencias = {}

# Iterar sobre os concelhos da base de dados 'Viver' e encontrar correspondências na base de dados 'Quebras'
for concelho_viver in concelhos_viver:
    # Encontrar a correspondência na base de dados 'Quebras'
    correspondencia_atual, pontuacao = encontrar_correspondencia(concelho_viver, concelhos_quebras)
    
    # Adicionar a correspondência ao dicionário de mapeamento
    mapeamento_correspondencias[concelho_viver] = (correspondencia_atual, pontuacao)

# Atualizar a base de dados 'Viver' com as correspondências encontradas
for concelho_viver, (correspondencia_atual, pontuacao) in mapeamento_correspondencias.items():
    Viver.loc[Viver['Concelho'] == concelho_viver, 'Concelho'] = correspondencia_atual
    
    # Exibir a correspondência e a pontuação de similaridade
    print(f"Correspondência de '{concelho_viver}' para -> {correspondencia_atual} (Pontuação: {pontuacao})")

# Resultado final
Viver


Correspondência de 'Arcos De Valdevez' para -> Arcos De Valdevez (Pontuação: 100)
Correspondência de 'Caminha' para -> Caminha (Pontuação: 100)
Correspondência de 'Melgaço' para -> Melgaço (Pontuação: 100)
Correspondência de 'Monção' para -> Monção (Pontuação: 100)
Correspondência de 'Paredes De Coura' para -> Paredes De Coura (Pontuação: 100)
Correspondência de 'Ponte Da Barca' para -> Ponte Da Barca (Pontuação: 100)
Correspondência de 'Ponte De Lima' para -> Ponte De Lima (Pontuação: 100)
Correspondência de 'Valença' para -> Valença (Pontuação: 100)
Correspondência de 'Viana Do Castelo' para -> Viana Do Castelo (Pontuação: 100)
Correspondência de 'Vila Nova De Cerveira' para -> Vila Nova De Cerveira (Pontuação: 100)
Correspondência de 'Amares' para -> Amares (Pontuação: 100)
Correspondência de 'Barcelos' para -> Barcelos (Pontuação: 100)
Correspondência de 'Braga' para -> Braga (Pontuação: 100)
Correspondência de 'Esposende' para -> Esposende (Pontuação: 100)
Correspondência de 'Terr

,Concelho,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos De Valdevez,0,24686,24603,24448,24259,24062,23854,23692,23519,...,21944,21664,21420,21227,21076,20926,20844,20826,20754,20693
1,Caminha,0,17055,17100,17088,17051,17013,16986,16959,16892,...,16500,16345,16177,16045,15940,15873,15848,15863,15917,16033
2,Melgaço,0,9971,9949,9889,9796,9715,9630,9538,9465,...,8785,8652,8531,8418,8279,8136,8029,7901,7755,7648
3,Monção,0,19925,19950,19919,19843,19773,19698,19615,19536,...,18782,18556,18361,18199,18063,17953,17890,17909,17898,17868
4,Paredes De Coura,0,9565,9556,9509,9459,9429,9404,9368,9321,...,8952,8869,8811,8761,8706,8677,8685,8711,8703,8676
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Braga,0,12523,12683,12827,12971,13087,13165,13247,13316,...,13158,12969,12765,12625,12570,12585,12679,12749,12788,12841
304,Santa Maria Da Feira,0,30301,31734,33120,34466,35805,37154,38512,39847,...,42685,42307,41932,41690,41602,41625,41753,42115,42666,43190
305,Santarém,0,8750,8685,8605,8511,8395,8276,8169,8062,...,7357,7204,7049,6911,6800,6722,6677,6624,6554,6490
306,São João Da Pesqueira,0,6179,6174,6138,6078,6020,5979,5946,5904,...,5398,5255,5124,5027,4949,4897,4867,4872,4875,4861


In [38]:
for index, row in Quebras.iterrows():
    # Obtendo o ano e o concelho da linha atual
    ano = row['Ano']
    if ano == 2023:
        ano = 2022
    concelho = row['Concelho']
    
    # Verificando se o ano existe como coluna na tabela 'Denso'
    if ano in Viver.columns:
        # Verificando se o concelho está presente na tabela 'Denso'
        if concelho in Viver['Concelho'].values:
            # Filtrando a linha correspondente na tabela 'Denso' com base no concelho
            linha_denso = Viver[Viver['Concelho'] == concelho]
            
            # Obtendo o valor para o ano desejado
            valor = linha_denso[ano].values[0]
        else:
            # Atribuindo valor nulo se o concelho não estiver presente na tabela 'Denso'
            valor = None
    else:
        # Atribuindo valor nulo se o ano não estiver presente como coluna na tabela 'Denso'
        valor = None
    
    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'População Residente'] = valor

Quebras


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,"Militar, Forças de Segurança e Forças de Socorro",Museus e Galerias de Arte,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos,Densidade Populacional,População Residente
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,0.0,0.0,0.0,0.0,0.0,13.0,33.0,2443.0,2516.2,176158.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.1,14792.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,0.0,0.0,0.0,0.0,0.0,3.0,6.0,439.0,486.5,56307.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,0.0,0.0,0.0,0.0,0.0,6.0,18.0,1358.0,198.6,80859.0
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,0.0,0.0,0.0,0.0,0.0,1.0,4.0,361.0,10.9,6256.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,0.0,0.0,0.0,0.0,0.0,4.0,7.0,429.0,369.4,54559.0
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,0.0,0.0,1.0,1.0,1.0,11.0,19.0,1002.0,631.8,136385.0
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,0.0,0.0,1.0,1.0,1.0,11.0,19.0,1002.0,631.8,136385.0
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,3.0,120.0,100.7,22374.0


## 9.5. Setores de Atividade

In [39]:
from fuzzywuzzy import process

# Lista de bases de dados e seus respectivos nomes
bases_de_dados = [WorkP, WorkS, WorkT]
nomes_bases_de_dados = ['WorkP', 'WorkS', 'WorkT']

# Lista de concelhos da base de dados 'Quebras'
concelhos_quebras = Quebras['Concelho'].unique()

# Função para encontrar a correspondência fuzzy entre os concelhos
def encontrar_correspondencia(concelho_quebras, concelhos_bd_atual):
    melhor_correspondencia, pontuacao = process.extractOne(concelho_quebras, concelhos_bd_atual)
    return melhor_correspondencia, pontuacao

# Iterar sobre as bases de dados
for idx, bd in enumerate(bases_de_dados):
    # Lista de concelhos da base de dados atual
    concelhos_bd_atual = bd['Concelho'].unique()
    
    # Criar um dicionário de mapeamento de correspondências fuzzy
    mapeamento_correspondencias = {}
    
    # Iterar sobre os concelhos da base de dados 'Quebras' e encontrar correspondências nas outras bases de dados
    for concelho_quebras in concelhos_quebras:
        # Encontrar a correspondência na base de dados atual
        correspondencia_atual, pontuacao = encontrar_correspondencia(concelho_quebras, concelhos_bd_atual)
        
        # Adicionar a correspondência ao dicionário de mapeamento
        mapeamento_correspondencias[concelho_quebras] = (correspondencia_atual, pontuacao)
        
    # Atualizar a base de dados atual com as correspondências encontradas
    for concelho_quebras, (correspondencia_atual, pontuacao) in mapeamento_correspondencias.items():
        if pontuacao >= 90:
            bd.loc[bd['Concelho'] == correspondencia_atual, 'Concelho'] = concelho_quebras
            
            # Exibir a correspondência e a pontuação de similaridade
            print(f"Correspondência de '{concelho_quebras}' na base de dados {nomes_bases_de_dados[idx]} para -> {correspondencia_atual} (Pontuação: {pontuacao})")
        
    print(f"\nCorrespondências atualizadas com sucesso na base de dados {nomes_bases_de_dados[idx]}\n")

# Resultado final
WorkP
WorkS
WorkT

Correspondência de 'Almada' na base de dados WorkP para -> Almada (Pontuação: 100)
Correspondência de 'Sertã' na base de dados WorkP para -> Sertã (Pontuação: 100)
Correspondência de 'Felgueiras' na base de dados WorkP para -> Felgueiras (Pontuação: 100)
Correspondência de 'Torres Vedras' na base de dados WorkP para -> Torres Vedras (Pontuação: 100)
Correspondência de 'Nisa' na base de dados WorkP para -> Nisa (Pontuação: 100)
Correspondência de 'Esposende' na base de dados WorkP para -> Esposende (Pontuação: 100)
Correspondência de 'Idanha-A-Nova' na base de dados WorkP para -> Idanha-A-Nova (Pontuação: 100)
Correspondência de 'Arraiolos' na base de dados WorkP para -> Arraiolos (Pontuação: 100)
Correspondência de 'Amarante' na base de dados WorkP para -> Amarante (Pontuação: 100)
Correspondência de 'Setúbal' na base de dados WorkP para -> Setúbal (Pontuação: 100)
Correspondência de 'Lamego' na base de dados WorkP para -> Lamego (Pontuação: 100)
Correspondência de 'Águeda' na base de 

,Concelho,2011,2021
0,Arcos De Valdevez,4175,4141
1,Caminha,4386,4438
2,Melgaço,1767,1657
3,Monção,4077,4182
4,Paredes De Coura,1910,1750
...,...,...,...
303,Ribeira Brava,3454,3441
304,Santa Cruz,16902,16317
305,Santana,1855,1722
306,São Vicente,1351,1281


### 9.5.1. Primário

In [40]:
for index, row in Quebras.iterrows():
    # Obtendo o ano da linha atual de 'Quebras'
    ano_quebras = row['Ano']
    
    # Obtendo o concelho da linha atual de 'Quebras'
    concelho = row['Concelho']
    
    # Encontrando o ano mais próximo na tabela 'WorkP'
    anos_workp = [int(col) for col in WorkP.columns[1:]]  # Convertendo os anos para inteiros
    ano_proximo = min(anos_workp, key=lambda x: abs(x - ano_quebras))

    # Obtendo o índice da linha correspondente ao concelho na tabela 'WorkP'
    idx_concelho = WorkP.index[WorkP['Concelho'] == concelho].tolist()[0]

    # Obtendo o valor da coluna correspondente ao ano mais próximo
    valor = WorkP.loc[idx_concelho, ano_proximo]

    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'Setor Primário'] = valor

Quebras


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Museus e Galerias de Arte,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos,Densidade Populacional,População Residente,Setor Primário
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,0.0,0.0,0.0,0.0,13.0,33.0,2443.0,2516.2,176158.0,384.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.1,14792.0,348.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,0.0,0.0,0.0,0.0,3.0,6.0,439.0,486.5,56307.0,384.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,0.0,0.0,0.0,0.0,6.0,18.0,1358.0,198.6,80859.0,2672.0
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,0.0,0.0,0.0,0.0,1.0,4.0,361.0,10.9,6256.0,165.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,0.0,0.0,0.0,0.0,4.0,7.0,429.0,369.4,54559.0,290.0
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,0.0,1.0,1.0,1.0,11.0,19.0,1002.0,631.8,136385.0,405.0
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,0.0,1.0,1.0,1.0,11.0,19.0,1002.0,631.8,136385.0,405.0
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,0.0,0.0,0.0,0.0,1.0,3.0,120.0,100.7,22374.0,1148.0


### 9.5.2. Secundário

In [41]:
for index, row in Quebras.iterrows():
    # Obtendo o ano da linha atual de 'Quebras'
    ano_quebras = row['Ano']
    
    # Obtendo o concelho da linha atual de 'Quebras'
    concelho = row['Concelho']
    
    # Encontrando o ano mais próximo na tabela 'WorkS'
    anos_works = [int(col) for col in WorkS.columns[1:]]  # Convertendo os anos para inteiros
    ano_proximo = min(anos_works, key=lambda x: abs(x - ano_quebras))

    # Obtendo o índice da linha correspondente ao concelho na tabela 'WorkS'
    idx_concelho = WorkS.index[WorkS['Concelho'] == concelho].tolist()[0]

    # Obtendo o valor da coluna correspondente ao ano mais próximo
    valor = WorkS.loc[idx_concelho, ano_proximo]

    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'Setor Secundário'] = valor

Quebras


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Parque Escolar,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos,Densidade Populacional,População Residente,Setor Primário,Setor Secundário
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,0.0,0.0,0.0,13.0,33.0,2443.0,2516.2,176158.0,384.0,10044.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,0.0,0.0,0.0,0.0,0.0,0.0,33.1,14792.0,348.0,1521.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,0.0,0.0,0.0,3.0,6.0,439.0,486.5,56307.0,384.0,16110.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,0.0,0.0,0.0,6.0,18.0,1358.0,198.6,80859.0,2672.0,9211.0
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,0.0,0.0,0.0,1.0,4.0,361.0,10.9,6256.0,165.0,385.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,0.0,0.0,0.0,4.0,7.0,429.0,369.4,54559.0,290.0,10052.0
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,1.0,1.0,1.0,11.0,19.0,1002.0,631.8,136385.0,405.0,27541.0
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,1.0,1.0,1.0,11.0,19.0,1002.0,631.8,136385.0,405.0,27541.0
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,0.0,0.0,0.0,1.0,3.0,120.0,100.7,22374.0,1148.0,1863.0


### 9.5.3. Terciário

In [42]:
for index, row in Quebras.iterrows():
    # Obtendo o ano da linha atual de 'Quebras'
    ano_quebras = row['Ano']
    
    # Obtendo o concelho da linha atual de 'Quebras'
    concelho = row['Concelho']
    
    # Encontrando o ano mais próximo na tabela 'WorkT'
    anos_workt = [int(col) for col in WorkT.columns[1:]]  # Convertendo os anos para inteiros
    ano_proximo = min(anos_workt, key=lambda x: abs(x - ano_quebras))

    # Obtendo o índice da linha correspondente ao concelho na tabela 'WorkT'
    idx_concelho = WorkT.index[WorkT['Concelho'] == concelho].tolist()[0]

    # Obtendo o valor da coluna correspondente ao ano mais próximo
    valor = WorkT.loc[idx_concelho, ano_proximo]

    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'Setor Terciário'] = valor

Quebras


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Serviços Administrativos,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos,Densidade Populacional,População Residente,Setor Primário,Setor Secundário,Setor Terciário
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,0.0,0.0,13.0,33.0,2443.0,2516.2,176158.0,384.0,10044.0,63062.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,0.0,0.0,0.0,0.0,0.0,33.1,14792.0,348.0,1521.0,3677.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,0.0,0.0,3.0,6.0,439.0,486.5,56307.0,384.0,16110.0,10154.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,0.0,0.0,6.0,18.0,1358.0,198.6,80859.0,2672.0,9211.0,25840.0
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,0.0,0.0,1.0,4.0,361.0,10.9,6256.0,165.0,385.0,1510.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,0.0,0.0,4.0,7.0,429.0,369.4,54559.0,290.0,10052.0,14465.0
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,1.0,1.0,11.0,19.0,1002.0,631.8,136385.0,405.0,27541.0,33907.0
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,1.0,1.0,11.0,19.0,1002.0,631.8,136385.0,405.0,27541.0,33907.0
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,0.0,0.0,1.0,3.0,120.0,100.7,22374.0,1148.0,1863.0,6030.0


## 9.6. Indice de Envelhecimento

In [43]:
Velho

,Concelho,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos De Valdevez,0,208.1,216.4,223.7,231.3,238.7,244,248.3,253.1,...,273.1,282.8,296,310.1,324.8,339.6,355.1,369.2,379.3,391.6
1,Caminha,0,138.1,141.1,145,147.3,149.9,153.1,156.4,161.3,...,203.4,209.8,218.9,229.2,239.7,247.6,253.3,258.9,265.8,272.3
2,Melgaço,0,297.4,305.3,311.1,321.1,332.3,348.2,367.2,376.7,...,393.4,410.7,433.9,451.1,473.6,501,526.9,548.2,563.3,577
3,Monção,0,209.7,217.6,222.3,227.2,234.8,242,246.3,246.3,...,277.2,284.3,292.3,301,310.2,318.2,324.1,332.2,343.5,351.5
4,Paredes De Coura,0,204.6,208.8,211.6,214.2,216.5,218.1,216.2,213.4,...,219,226,232.5,237.7,244.3,250.3,256.3,267.3,278.4,289.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,0,80.9,81,78.5,76.5,76.1,76,76,76.9,...,96.4,102.2,108,114.8,122.5,131.4,141.7,152,162.1,169.5
304,Santa Cruz,0,56.1,55.4,54.4,53.1,52.1,51.2,50.4,49.7,...,57.9,60.8,63.9,67.8,72.7,78.1,83.8,89.9,95.7,101.1
305,Santana,0,128.1,136.8,143.8,149,151.7,153.7,159.1,165.3,...,189.7,201.1,211.2,222.4,234.8,242.8,253,265.4,279.7,290.4
306,São Vicente,0,121.5,127,131.1,133.7,136.7,141.1,146,152,...,169.4,180.6,190.5,197.8,206,215.3,228.4,243.7,258.3,273.9


In [44]:
from fuzzywuzzy import process

# Lista de concelhos do DataFrame 'Velho'
concelhos_velho = Velho['Concelho'].unique()

# Lista de concelhos da base de dados 'Quebras'
concelhos_quebras = Quebras['Concelho'].unique()

# Função para encontrar a correspondência fuzzy entre os concelhos
def encontrar_correspondencia(concelho_velho, concelhos_quebras):
    melhor_correspondencia, pontuacao = process.extractOne(concelho_velho, concelhos_quebras)
    return melhor_correspondencia, pontuacao

# Criar um dicionário de mapeamento de correspondências fuzzy
mapeamento_correspondencias = {}

# Iterar sobre os concelhos do DataFrame 'Velho' e encontrar correspondências na base de dados 'Quebras'
for concelho_velho in concelhos_velho:
    # Encontrar a correspondência na base de dados 'Quebras'
    correspondencia_atual, pontuacao = encontrar_correspondencia(concelho_velho, concelhos_quebras)
    
    # Adicionar a correspondência ao dicionário de mapeamento
    mapeamento_correspondencias[concelho_velho] = (correspondencia_atual, pontuacao)

# Atualizar o DataFrame 'Velho' com as correspondências encontradas
for concelho_velho, (correspondencia_atual, pontuacao) in mapeamento_correspondencias.items():
    if pontuacao >= 90:
        Velho.loc[Velho['Concelho'] == concelho_velho, 'Concelho'] = correspondencia_atual
        
        # Exibir a correspondência e a pontuação de similaridade
        print(f"Correspondência de '{concelho_velho}' para -> {correspondencia_atual} (Pontuação: {pontuacao})")

# Resultado final
Velho

Correspondência de 'Arcos De Valdevez' para -> Arcos De Valdevez (Pontuação: 100)
Correspondência de 'Caminha' para -> Caminha (Pontuação: 100)
Correspondência de 'Melgaço' para -> Melgaço (Pontuação: 100)
Correspondência de 'Monção' para -> Monção (Pontuação: 100)
Correspondência de 'Paredes De Coura' para -> Paredes De Coura (Pontuação: 100)
Correspondência de 'Ponte Da Barca' para -> Ponte Da Barca (Pontuação: 100)
Correspondência de 'Ponte De Lima' para -> Ponte De Lima (Pontuação: 100)
Correspondência de 'Valença' para -> Valença (Pontuação: 100)
Correspondência de 'Viana Do Castelo' para -> Viana Do Castelo (Pontuação: 100)
Correspondência de 'Vila Nova De Cerveira' para -> Vila Nova De Cerveira (Pontuação: 100)
Correspondência de 'Amares' para -> Amares (Pontuação: 100)
Correspondência de 'Barcelos' para -> Barcelos (Pontuação: 100)
Correspondência de 'Braga' para -> Braga (Pontuação: 100)
Correspondência de 'Esposende' para -> Esposende (Pontuação: 100)
Correspondência de 'Terr

,Concelho,2000,2001,2002,2003,2004,2005,2006,2007,2008,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Arcos De Valdevez,0,208.1,216.4,223.7,231.3,238.7,244,248.3,253.1,...,273.1,282.8,296,310.1,324.8,339.6,355.1,369.2,379.3,391.6
1,Caminha,0,138.1,141.1,145,147.3,149.9,153.1,156.4,161.3,...,203.4,209.8,218.9,229.2,239.7,247.6,253.3,258.9,265.8,272.3
2,Melgaço,0,297.4,305.3,311.1,321.1,332.3,348.2,367.2,376.7,...,393.4,410.7,433.9,451.1,473.6,501,526.9,548.2,563.3,577
3,Monção,0,209.7,217.6,222.3,227.2,234.8,242,246.3,246.3,...,277.2,284.3,292.3,301,310.2,318.2,324.1,332.2,343.5,351.5
4,Paredes De Coura,0,204.6,208.8,211.6,214.2,216.5,218.1,216.2,213.4,...,219,226,232.5,237.7,244.3,250.3,256.3,267.3,278.4,289.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Ribeira Brava,0,80.9,81,78.5,76.5,76.1,76,76,76.9,...,96.4,102.2,108,114.8,122.5,131.4,141.7,152,162.1,169.5
304,Santa Cruz,0,56.1,55.4,54.4,53.1,52.1,51.2,50.4,49.7,...,57.9,60.8,63.9,67.8,72.7,78.1,83.8,89.9,95.7,101.1
305,Santana,0,128.1,136.8,143.8,149,151.7,153.7,159.1,165.3,...,189.7,201.1,211.2,222.4,234.8,242.8,253,265.4,279.7,290.4
306,São Vicente,0,121.5,127,131.1,133.7,136.7,141.1,146,152,...,169.4,180.6,190.5,197.8,206,215.3,228.4,243.7,258.3,273.9


In [45]:
for index, row in Quebras.iterrows():
    # Obtendo o ano e o concelho da linha atual
    ano = row['Ano']
    if ano == 2023:
        ano = 2022
    concelho = row['Concelho']
    
    # Verificando se o ano existe como coluna na tabela 'Denso'
    if ano in Velho.columns:
        # Verificando se o concelho está presente na tabela 'Denso'
        if concelho in Velho['Concelho'].values:
            # Filtrando a linha correspondente na tabela 'Denso' com base no concelho
            linha_denso = Velho[Velho['Concelho'] == concelho]
            
            # Obtendo o valor para o ano desejado
            valor = linha_denso[ano].values[0]
        else:
            # Atribuindo valor nulo se o concelho não estiver presente na tabela 'Denso'
            valor = None
    else:
        # Atribuindo valor nulo se o ano não estiver presente como coluna na tabela 'Denso'
        valor = None
    
    # Atualizando a linha 'Quebras' com o valor obtido
    Quebras.at[index, 'Indice de Envelhecimento'] = valor

Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Áreas Comerciais e Gares de Transportes,Ocorrências,MeiosTerrestres,Duração Minutos,Densidade Populacional,População Residente,Setor Primário,Setor Secundário,Setor Terciário,Indice de Envelhecimento
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,0.0,13.0,33.0,2443.0,2516.2,176158.0,384.0,10044.0,63062.0,157.9
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,0.0,0.0,0.0,0.0,33.1,14792.0,348.0,1521.0,3677.0,260.2
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,0.0,3.0,6.0,439.0,486.5,56307.0,384.0,16110.0,10154.0,117.9
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,0.0,6.0,18.0,1358.0,198.6,80859.0,2672.0,9211.0,25840.0,156.4
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,0.0,1.0,4.0,361.0,10.9,6256.0,165.0,385.0,1510.0,467.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,0.0,4.0,7.0,429.0,369.4,54559.0,290.0,10052.0,14465.0,141.7
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,1.0,11.0,19.0,1002.0,631.8,136385.0,405.0,27541.0,33907.0,134.6
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,1.0,11.0,19.0,1002.0,631.8,136385.0,405.0,27541.0,33907.0,134.6
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,0.0,1.0,3.0,120.0,100.7,22374.0,1148.0,1863.0,6030.0,175.1


## 9.7. tx Ruralidade

In [46]:
# Lista de concelhos do DataFrame 'ZonasRurais'
distrito_ZonasRurais = ZonasRurais['Distrito'].unique()

# Lista de concelhos da base de dados 'Quebras'
distritos_quebras = Quebras['Distrito'].unique()

# Função para encontrar a correspondência fuzzy entre os concelhos
def encontrar_correspondencia(distrito_ZonasRurais, distritos_quebras):
    melhor_correspondencia, pontuacao = process.extractOne(distrito_ZonasRurais, distritos_quebras)
    return melhor_correspondencia, pontuacao

# Criar um dicionário de mapeamento de correspondências fuzzy
mapeamento_correspondencias = {}

# Iterar sobre os concelhos do DataFrame 'ZonasRurais' e encontrar correspondências na base de dados 'Quebras'
for distrito_ZonasRurais in distrito_ZonasRurais:
    # Encontrar a correspondência na base de dados 'Quebras'
    correspondencia_atual, pontuacao = encontrar_correspondencia(distrito_ZonasRurais, distritos_quebras)
    
    # Adicionar a correspondência ao dicionário de mapeamento
    mapeamento_correspondencias[distrito_ZonasRurais] = (correspondencia_atual, pontuacao)

# Atualizar o DataFrame 'ZonasRurais' com as correspondências encontradas
for distrito_ZonasRurais, (correspondencia_atual, pontuacao) in mapeamento_correspondencias.items():
    if pontuacao >= 90:
        ZonasRurais.loc[ZonasRurais['Concelho'] == distrito_ZonasRurais, 'Concelho'] = correspondencia_atual
        
        # Exibir a correspondência e a pontuação de similaridade
        print(f"Correspondência de '{distrito_ZonasRurais}' para -> {correspondencia_atual} (Pontuação: {pontuacao})")

# Resultado final
ZonasRurais

Correspondência de 'Aveiro' para -> Aveiro (Pontuação: 100)
Correspondência de 'Beja' para -> Beja (Pontuação: 100)
Correspondência de 'Braga' para -> Braga (Pontuação: 100)
Correspondência de 'Bragança' para -> Bragança (Pontuação: 100)
Correspondência de 'Castelo Branco' para -> Castelo Branco (Pontuação: 100)
Correspondência de 'Coimbra' para -> Coimbra (Pontuação: 100)
Correspondência de 'Faro' para -> Faro (Pontuação: 100)
Correspondência de 'Guarda' para -> Guarda (Pontuação: 100)
Correspondência de 'Leiria' para -> Leiria (Pontuação: 100)
Correspondência de 'Lisboa' para -> Lisboa (Pontuação: 100)
Correspondência de 'Portalegre' para -> Portalegre (Pontuação: 100)
Correspondência de 'Porto' para -> Porto (Pontuação: 100)
Correspondência de 'Santarém' para -> Santarém (Pontuação: 100)
Correspondência de 'Setúbal' para -> Setúbal (Pontuação: 100)
Correspondência de 'Viana Do Castelo' para -> Viana Do Castelo (Pontuação: 100)
Correspondência de 'Vila Real' para -> Vila Real (Pontua

,Distrito,Concelho,Zona Rural,Zona Urbana,Rural
0,Aveiro,Albergaria-A-Velha,6,0,1.00
1,Aveiro,Anadia,10,0,1.00
2,Aveiro,Arouca,16,0,1.00
3,Aveiro,Aveiro,0,10,0.00
4,Aveiro,Castelo De Paiva,6,0,1.00
...,...,...,...,...,...
273,Évora,Reguengos De Monsaraz,4,0,1.00
274,Évora,Vendas Novas,2,0,1.00
275,Évora,Viana Do Alentejo,3,0,1.00
276,Évora,Vila Viçosa,4,0,1.00


In [47]:
# Lista de distritos do DataFrame 'ZonasRurais'
distritos_zonas_rurais = ZonasRurais['Distrito'].unique()

# Criar um dicionário de mapeamento de correspondências fuzzy
mapeamento_correspondencias = {}

# Iterar sobre os distritos do DataFrame 'ZonasRurais'
for distrito in distritos_zonas_rurais:
    # Filtrar os concelhos da base de dados 'Quebras' pelo distrito atual
    concelhos_quebras_distrito = Quebras[Quebras['Distrito'] == distrito]['Concelho'].unique()
    
    # Iterar sobre os concelhos Distrito atual do DataFrame 'ZonasRurais'
    for concelho_zonas_rurais in ZonasRurais[ZonasRurais['Distrito'] == distrito]['Concelho'].unique():
        # Encontrar a correspondência fuzzy para o concelho atual
        correspondencia, pontuacao = process.extractOne(concelho_zonas_rurais, concelhos_quebras_distrito)
        
        # Adicionar a correspondência ao dicionário de mapeamento
        mapeamento_correspondencias[(distrito, concelho_zonas_rurais)] = (correspondencia, pontuacao)

# Atualizar o DataFrame 'ZonasRurais' com as correspondências encontradas
for (distrito, concelho), (correspondencia, pontuacao) in mapeamento_correspondencias.items():
    if pontuacao >= 90:
        # Filtrar o DataFrame 'ZonasRurais' pelo distrito e pelo concelho atual
        filtro = (ZonasRurais['Distrito'] == distrito) & (ZonasRurais['Concelho'] == concelho)
        
        # Atualizar a coluna 'Concelho' com a correspondência encontrada
        ZonasRurais.loc[filtro, 'Concelho'] = correspondencia
        
        # Exibir a correspondência e a pontuação de similaridade
        print(f"Correspondência de '{concelho}' no distrito '{distrito}' para -> '{correspondencia}' (Pontuação: {pontuacao})")

# Resultado final
ZonasRurais


Correspondência de 'Albergaria-A-Velha' no distrito 'Aveiro' para -> 'Albergaria-A-Velha' (Pontuação: 100)
Correspondência de 'Anadia' no distrito 'Aveiro' para -> 'Anadia' (Pontuação: 100)
Correspondência de 'Arouca' no distrito 'Aveiro' para -> 'Arouca' (Pontuação: 100)
Correspondência de 'Aveiro' no distrito 'Aveiro' para -> 'Aveiro' (Pontuação: 100)
Correspondência de 'Castelo De Paiva' no distrito 'Aveiro' para -> 'Castelo De Paiva' (Pontuação: 100)
Correspondência de 'Espinho' no distrito 'Aveiro' para -> 'Espinho' (Pontuação: 100)
Correspondência de 'Estarreja' no distrito 'Aveiro' para -> 'Estarreja' (Pontuação: 100)
Correspondência de 'Mealhada' no distrito 'Aveiro' para -> 'Mealhada' (Pontuação: 100)
Correspondência de 'Murtosa' no distrito 'Aveiro' para -> 'Murtosa' (Pontuação: 100)
Correspondência de 'Oliveira De Azeméis' no distrito 'Aveiro' para -> 'Oliveira De Azeméis' (Pontuação: 100)
Correspondência de 'Oliveira Do Bairro' no distrito 'Aveiro' para -> 'Oliveira Do Bair

,Distrito,Concelho,Zona Rural,Zona Urbana,Rural
0,Aveiro,Albergaria-A-Velha,6,0,1.00
1,Aveiro,Anadia,10,0,1.00
2,Aveiro,Arouca,16,0,1.00
3,Aveiro,Aveiro,0,10,0.00
4,Aveiro,Castelo De Paiva,6,0,1.00
...,...,...,...,...,...
273,Évora,Reguengos De Monsaraz,4,0,1.00
274,Évora,Vendas Novas,2,0,1.00
275,Évora,Viana Do Alentejo,3,0,1.00
276,Évora,Vila Viçosa,4,0,1.00


In [48]:
Quebras = Quebras.merge(ZonasRurais[['Concelho', 'Rural']], on='Concelho', how='left')
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Ocorrências,MeiosTerrestres,Duração Minutos,Densidade Populacional,População Residente,Setor Primário,Setor Secundário,Setor Terciário,Indice de Envelhecimento,Rural
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,13.0,33.0,2443.0,2516.2,176158.0,384.0,10044.0,63062.0,157.9,0.000000
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,0.0,0.0,0.0,33.1,14792.0,348.0,1521.0,3677.0,260.2,1.000000
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,3.0,6.0,439.0,486.5,56307.0,384.0,16110.0,10154.0,117.9,0.800000
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,6.0,18.0,1358.0,198.6,80859.0,2672.0,9211.0,25840.0,156.4,0.923077
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,1.0,4.0,361.0,10.9,6256.0,165.0,385.0,1510.0,467.9,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,4.0,7.0,429.0,369.4,54559.0,290.0,10052.0,14465.0,141.7,1.000000
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,11.0,19.0,1002.0,631.8,136385.0,405.0,27541.0,33907.0,134.6,0.000000
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,11.0,19.0,1002.0,631.8,136385.0,405.0,27541.0,33907.0,134.6,0.000000
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,1.0,3.0,120.0,100.7,22374.0,1148.0,1863.0,6030.0,175.1,1.000000


## 9.8. IPMA

In [49]:
# Lista de todas as tabelas de clima
tabelas_clima = tabelas = [('T_MED', T_MED), ('T_MAX', T_MAX), ('T_MIN', T_MIN), ('DD_MED', DD_MED), ('DD_FFX', DD_FFX), ('FF_MED', FF_MED), ('FF_MAX', FF_MAX), ('PR_QTD', PR_QTD)]

# Iterar sobre todas as tabelas de clima
for tabela in tabelas_clima:
    # Obtendo o nome da tabela
    nome_tabela = tabela[0]
    TabeIPMA = tabela[1]
    # Iterar sobre as linhas de Quebras
    for index, row in Quebras.iterrows():

        ano = int(row['Ano'])
        mes = int(row['Mês'])
        dia = int(row['Dia'])
        concelho = row["Distrito"]

        
        valor = TabeIPMA[concelho][(TabeIPMA["Ano"] == ano) & (TabeIPMA["Mês"] == mes) & (TabeIPMA["Dia"] == dia)].iloc[0]
        
        # Atualizando a linha 'Quebras' com o valor obtido
        Quebras.at[index, f'{nome_tabela}'] = valor

# Exibindo o DataFrame Quebras atualizado
Quebras


,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Indice de Envelhecimento,Rural,T_MED,T_MAX,T_MIN,DD_MED,DD_FFX,FF_MED,FF_MAX,PR_QTD
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,157.9,0.000000,10.7,17.4,2.6,-990.0,-990.0,-990.0,-990.0,0.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,260.2,1.000000,10.6,13.8,6.2,249.0,270.0,2.8,8.1,0.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,117.9,0.800000,14.0,14.8,12.9,197.0,180.0,3.9,14.4,4.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,156.4,0.923077,12.4,15.1,9.1,288.0,225.0,3.6,13.9,13.2
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,467.9,1.000000,4.9,7.8,2.2,137.0,118.0,4.1,15.4,15.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,141.7,1.000000,10.3,14.3,6.3,-990.0,-990.0,-990.0,-990.0,0.0
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,134.6,0.000000,16.4,20.6,14.8,-990.0,-990.0,-990.0,-990.0,-990.0
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,134.6,0.000000,17.9,22.6,14.6,-990.0,-990.0,-990.0,-990.0,8.1
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,175.1,1.000000,15.0,20.8,12.8,186.0,186.0,2.5,11.0,-990.0


In [50]:
Quebras[Quebras.isna().any(axis=1)]

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Indice de Envelhecimento,Rural,T_MED,T_MAX,T_MIN,DD_MED,DD_FFX,FF_MED,FF_MAX,PR_QTD


In [51]:
Quebras.replace(-990.0, None, inplace=True)
Quebras

,Código do Relatório,Concelho,Data do incidente,Nível de Tensão,Causa do incidente,Duração incidente (min),N.º clientes afetados,TIEPI MT (min),SAIFI MT (#),SAIDI MT (min),...,Indice de Envelhecimento,Rural,T_MED,T_MAX,T_MIN,DD_MED,DD_FFX,FF_MED,FF_MAX,PR_QTD
0,EDPD_2018_JAN_T_1,Almada,2018-01-02,MT,Escavações,55,828,"0,0029345","0,0000443","0,0024563",...,157.9,0.000000,10.7,17.4,2.6,None,None,None,None,0.0
1,EDPD_2018_JAN_T_2,Sertã,2018-01-02,MT,Abate De Árvores,163,549,"0,0029335","0,0001770","0,0165009",...,260.2,1.000000,10.6,13.8,6.2,249.0,270.0,2.8,8.1,0.0
2,EDPD_2018_JAN_T_3,Felgueiras,2018-01-04,MT,Deslizamento De Terras,596,3459,"0,0192281","0,0002655","0,0359047",...,117.9,0.800000,14.0,14.8,12.9,197.0,180.0,3.9,14.4,4.0
3,EDPD_2018_JAN_T_4,Torres Vedras,2018-01-05,BT,Veículos,178,102,"0,0000000","0,0000000","0,0000000",...,156.4,0.923077,12.4,15.1,9.1,288.0,225.0,3.6,13.9,13.2
4,EDPD_2018_JAN_T_5,Nisa,2018-01-09,MT,Aves,119,1731,"0,0044221","0,0001549","0,0100377",...,467.9,1.000000,4.9,7.8,2.2,137.0,118.0,4.1,15.4,15.9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7723,EDPD_2017_NOV_T_62,Ovar,2017-11-29,BT,Veículos,220,2,0,0,0,...,141.7,1.000000,10.3,14.3,6.3,None,None,None,None,0.0
7724,EDPD_2017_NOV_T_63,Santa Maria Da Feira,2017-11-03,BT,Veículos,92,1,0,0,0,...,134.6,0.000000,16.4,20.6,14.8,None,None,None,None,None
7725,EDPD_2017_NOV_T_64,Santa Maria Da Feira,2017-11-23,BT,Veículos,121,1,0,0,0,...,134.6,0.000000,17.9,22.6,14.6,None,None,None,None,8.1
7726,EDPD_2017_NOV_T_65,Almeirim,2017-11-03,BT,Descarga Atmosférica Directa,154,1,0,0,0,...,175.1,1.000000,15.0,20.8,12.8,186.0,186.0,2.5,11.0,None


In [52]:
Quebras.to_excel("QuebrasJuntoInc.xlsx", index=False)

- - -

## EXTRA - Nº Eventos

In [53]:
Quebras.columns

Index(['Código do Relatório', 'Concelho', 'Data do incidente',
       'Nível de Tensão', 'Causa do incidente', 'Duração incidente (min)',
       'N.º clientes afetados', 'TIEPI MT (min)', 'SAIFI MT (#)',
       'SAIDI MT (min)', 'END (MWh)', 'SAIFI BT (#)', 'SAIDI BT (min)',
       'Qualidade de Energia Elétrica', 'Qualidade de Serviço Comercial',
       'Fundamentação', 'Decisão', 'Dia', 'Mês', 'Ano', 'Distrito',
       'Bibliotecas e Arquivos', 'Desporto e Lazer',
       'Edifícios degradados ou devolutos', 'Espectáculos e Reuniões Públicas',
       'Estacionamento de superfície',
       'Estacionamento em profundidade ou silo', 'Habitacional',
       'Hospitalares e Lares de Idosos', 'Hotelaria e Restauração',
       'Indústria, Oficina e Armazém',
       'Militar, Forças de Segurança e Forças de Socorro',
       'Museus e Galerias de Arte', 'Parque Escolar',
       'Serviços Administrativos', 'Áreas Comerciais e Gares de Transportes',
       'Ocorrências', 'MeiosTerrestres', 'Duraç

In [54]:
QuebrasContar = Quebras[["Distrito", "Concelho", "Data do incidente", "Nível de Tensão"]].groupby(["Distrito", "Concelho", "Data do incidente", "Nível de Tensão"]).size().reset_index(name='Número de Eventos')
QuebrasContar

,Distrito,Concelho,Data do incidente,Nível de Tensão,Número de Eventos
0,Aveiro,Albergaria-A-Velha,2014-01-11,MT,1
1,Aveiro,Albergaria-A-Velha,2014-02-20,MT,1
2,Aveiro,Albergaria-A-Velha,2014-04-07,BT,1
3,Aveiro,Albergaria-A-Velha,2014-05-21,MT,1
4,Aveiro,Albergaria-A-Velha,2014-05-23,BT,1
...,...,...,...,...,...
7393,Évora,Évora,2022-03-22,MT,1
7394,Évora,Évora,2022-05-17,MT,1
7395,Évora,Évora,2022-10-14,BT,1
7396,Évora,Évora,2023-06-07,MT,1


In [55]:
QuebrasContar["Data do incidente"] = pd.to_datetime(QuebrasContar["Data do incidente"])
QuebrasContar.dtypes

Distrito                     object
Concelho                     object
Data do incidente    datetime64[ns]
Nível de Tensão              object
Número de Eventos             int64
dtype: object

In [56]:
QuebrasContar["Número de Eventos"].unique()

array([ 1,  2, 10,  4,  3,  7,  5], dtype=int64)

In [57]:
# Criar um DataFrame de referência com todas as datas no intervalo desejado
datas = pd.date_range(start='2014-01-01', end='2023-12-31', freq='D')
df_referencia = pd.DataFrame({'Data do incidente': datas})
df_referencia

,Data do incidente
0,2014-01-01
1,2014-01-02
2,2014-01-03
3,2014-01-04
4,2014-01-05
...,...
3647,2023-12-27
3648,2023-12-28
3649,2023-12-29
3650,2023-12-30


In [58]:
combinacoes = Quebras[['Concelho', 'Distrito']].drop_duplicates()
combinacoes

,Concelho,Distrito
0,Almada,Setúbal
1,Sertã,Castelo Branco
2,Felgueiras,Porto
3,Torres Vedras,Lisboa
4,Nisa,Portalegre
...,...,...
4629,Campo Maior,Portalegre
5703,Castelo De Paiva,Aveiro
5771,Vila Flor,Bragança
6418,Vimioso,Bragança


In [59]:
# Fazer a combinação entre o DataFrame de referência e as combinações de 'Concelho' e 'Distrito'
merged_df = pd.merge(df_referencia.assign(key=0), combinacoes.assign(key=0), on='key', how='outer')
merged_df

,Data do incidente,key,Concelho,Distrito
0,2014-01-01,0,Almada,Setúbal
1,2014-01-01,0,Sertã,Castelo Branco
2,2014-01-01,0,Felgueiras,Porto
3,2014-01-01,0,Torres Vedras,Lisboa
4,2014-01-01,0,Nisa,Portalegre
...,...,...,...,...
1007947,2023-12-31,0,Campo Maior,Portalegre
1007948,2023-12-31,0,Castelo De Paiva,Aveiro
1007949,2023-12-31,0,Vila Flor,Bragança
1007950,2023-12-31,0,Vimioso,Bragança


In [60]:
# Juntar os valores originais da coluna 'Número de Eventos' (presumo que 'QuebrasContarOriginal' seja o dataframe original com os dados)
merged_df = pd.merge(merged_df, QuebrasContar, on=['Concelho', 'Distrito', 'Data do incidente'], how='left', suffixes=('', '_original'))

# Preencher os valores nulos na coluna 'Número de Eventos' com 0
merged_df['Número de Eventos'] = merged_df['Número de Eventos'].fillna(0).astype(int)

# Preencher os valores nulos na coluna 'Número de Eventos' com 0
merged_df['Nível de Tensão'] = merged_df['Nível de Tensão'].fillna("Sem Eventos Excecionais")

# Remover colunas extras
merged_df.drop(['key'], axis=1, inplace=True)

# Visualizar DB
merged_df

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0
...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0


In [61]:
merged_df["Número de Eventos"].unique()

array([ 0,  1,  2,  3,  4,  5,  7, 10])

In [62]:
merged_df['Dia'] = merged_df['Data do incidente'].dt.day
merged_df['Mês'] = merged_df['Data do incidente'].dt.month
merged_df['Ano'] = merged_df['Data do incidente'].dt.year

merged_df

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014
...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023


In [63]:
for index, row in merged_df.iterrows():
    # Obtendo o ano e o concelho da linha atual
    ano = row['Ano']
    if ano == 2023:
        ano = 2022
    concelho = row['Concelho']
    
    # Verificando se o ano existe como coluna na tabela 'Denso'
    if ano in Denso.columns:
        # Verificando se o concelho está presente na tabela 'Denso'
        if concelho in Denso['Concelho'].values:
            # Filtrando a linha correspondente na tabela 'Denso' com base no concelho
            linha_denso = Denso[Denso['Concelho'] == concelho]
            
            # Obtendo o valor para o ano desejado
            valor = linha_denso[ano].values[0]
        else:
            # Atribuindo valor nulo se o concelho não estiver presente na tabela 'Denso'
            valor = None
    else:
        # Atribuindo valor nulo se o ano não estiver presente como coluna na tabela 'Denso'
        valor = None
    
    # Atualizando a linha 'merged_df' com o valor obtido
    merged_df.at[index, 'Densidade Populacional'] = valor

merged_df

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014,2486.8
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014,34.0
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014,493.1
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014,197.2
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014,12.0
...,...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023,32.3
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023,134.1
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023,22.7
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023,8.6


In [64]:
for index, row in merged_df.iterrows():
    # Obtendo o ano e o concelho da linha atual
    ano = row['Ano']
    if ano == 2023:
        ano = 2022
    concelho = row['Concelho']
    
    # Verificando se o ano existe como coluna na tabela 'Denso'
    if ano in Viver.columns:
        # Verificando se o concelho está presente na tabela 'Denso'
        if concelho in Viver['Concelho'].values:
            # Filtrando a linha correspondente na tabela 'Denso' com base no concelho
            linha_denso = Viver[Viver['Concelho'] == concelho]
            
            # Obtendo o valor para o ano desejado
            valor = linha_denso[ano].values[0]
        else:
            # Atribuindo valor nulo se o concelho não estiver presente na tabela 'Denso'
            valor = None
    else:
        # Atribuindo valor nulo se o ano não estiver presente como coluna na tabela 'Denso'
        valor = None
    
    # Atualizando a linha 'merged_df' com o valor obtido
    merged_df.at[index, 'População Residente'] = valor

merged_df

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional,População Residente
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014,2486.8,174592.0
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014,34.0,15188.0
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014,493.1,57074.0
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014,197.2,80291.0
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014,12.0,6929.0
...,...,...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023,32.3,7993.0
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023,134.1,15420.0
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023,22.7,6042.0
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023,8.6,4143.0


In [65]:
for index, row in merged_df.iterrows():
    # Obtendo o ano da linha atual de 'merged_df'
    ano_quebras = row['Ano']
    
    # Obtendo o concelho da linha atual de 'merged_df'
    concelho = row['Concelho']
    
    # Encontrando o ano mais próximo na tabela 'WorkP'
    anos_workp = [int(col) for col in WorkP.columns[1:]]  # Convertendo os anos para inteiros
    ano_proximo = min(anos_workp, key=lambda x: abs(x - ano_quebras))

    # Obtendo o índice da linha correspondente ao concelho na tabela 'WorkP'
    idx_concelho = WorkP.index[WorkP['Concelho'] == concelho].tolist()[0]

    # Obtendo o valor da coluna correspondente ao ano mais próximo
    valor = WorkP.loc[idx_concelho, ano_proximo]

    # Atualizando a linha 'merged_df' com o valor obtido
    merged_df.at[index, 'Setor Primário'] = valor

merged_df


,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional,População Residente,Setor Primário
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014,2486.8,174592.0,492.0
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014,34.0,15188.0,373.0
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014,493.1,57074.0,445.0
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014,197.2,80291.0,2152.0
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014,12.0,6929.0,135.0
...,...,...,...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023,32.3,7993.0,229.0
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023,134.1,15420.0,190.0
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023,22.7,6042.0,410.0
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023,8.6,4143.0,168.0


In [66]:
for index, row in merged_df.iterrows():
    # Obtendo o ano da linha atual de 'merged_df'
    ano_quebras = row['Ano']
    
    # Obtendo o concelho da linha atual de 'merged_df'
    concelho = row['Concelho']
    
    # Encontrando o ano mais próximo na tabela 'WorkS'
    anos_works = [int(col) for col in WorkS.columns[1:]]  # Convertendo os anos para inteiros
    ano_proximo = min(anos_works, key=lambda x: abs(x - ano_quebras))

    # Obtendo o índice da linha correspondente ao concelho na tabela 'WorkS'
    idx_concelho = WorkS.index[WorkS['Concelho'] == concelho].tolist()[0]

    # Obtendo o valor da coluna correspondente ao ano mais próximo
    valor = WorkS.loc[idx_concelho, ano_proximo]

    # Atualizando a linha 'merged_df' com o valor obtido
    merged_df.at[index, 'Setor Secundário'] = valor

merged_df

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional,População Residente,Setor Primário,Setor Secundário
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014,2486.8,174592.0,492.0,10808.0
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014,34.0,15188.0,373.0,1776.0
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014,493.1,57074.0,445.0,16051.0
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014,197.2,80291.0,2152.0,9312.0
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014,12.0,6929.0,135.0,497.0
...,...,...,...,...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023,32.3,7993.0,229.0,1004.0
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023,134.1,15420.0,190.0,3155.0
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023,22.7,6042.0,410.0,384.0
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023,8.6,4143.0,168.0,233.0


In [67]:
for index, row in merged_df.iterrows():
    # Obtendo o ano da linha atual de 'merged_df'
    ano_quebras = row['Ano']
    
    # Obtendo o concelho da linha atual de 'merged_df'
    concelho = row['Concelho']
    
    # Encontrando o ano mais próximo na tabela 'WorkT'
    anos_workt = [int(col) for col in WorkT.columns[1:]]  # Convertendo os anos para inteiros
    ano_proximo = min(anos_workt, key=lambda x: abs(x - ano_quebras))

    # Obtendo o índice da linha correspondente ao concelho na tabela 'WorkT'
    idx_concelho = WorkT.index[WorkT['Concelho'] == concelho].tolist()[0]

    # Obtendo o valor da coluna correspondente ao ano mais próximo
    valor = WorkT.loc[idx_concelho, ano_proximo]

    # Atualizando a linha 'merged_df' com o valor obtido
    merged_df.at[index, 'Setor Terciário'] = valor

merged_df

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional,População Residente,Setor Primário,Setor Secundário,Setor Terciário
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014,2486.8,174592.0,492.0,10808.0,59539.0
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014,34.0,15188.0,373.0,1776.0,3538.0
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014,493.1,57074.0,445.0,16051.0,10220.0
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014,197.2,80291.0,2152.0,9312.0,23406.0
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014,12.0,6929.0,135.0,497.0,1675.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023,32.3,7993.0,229.0,1004.0,2126.0
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023,134.1,15420.0,190.0,3155.0,3154.0
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023,22.7,6042.0,410.0,384.0,1297.0
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023,8.6,4143.0,168.0,233.0,896.0


In [68]:
for index, row in merged_df.iterrows():
    # Obtendo o ano e o concelho da linha atual
    ano = row['Ano']
    if ano == 2023:
        ano = 2022
    concelho = row['Concelho']
    
    # Verificando se o ano existe como coluna na tabela 'Denso'
    if ano in Velho.columns:
        # Verificando se o concelho está presente na tabela 'Denso'
        if concelho in Velho['Concelho'].values:
            # Filtrando a linha correspondente na tabela 'Denso' com base no concelho
            linha_denso = Velho[Velho['Concelho'] == concelho]
            
            # Obtendo o valor para o ano desejado
            valor = linha_denso[ano].values[0]
        else:
            # Atribuindo valor nulo se o concelho não estiver presente na tabela 'Denso'
            valor = None
    else:
        # Atribuindo valor nulo se o ano não estiver presente como coluna na tabela 'Denso'
        valor = None
    
    # Atualizando a linha 'merged_df' com o valor obtido
    merged_df.at[index, 'Indice de Envelhecimento'] = valor

merged_df

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional,População Residente,Setor Primário,Setor Secundário,Setor Terciário,Indice de Envelhecimento
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014,2486.8,174592.0,492.0,10808.0,59539.0,142.4
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014,34.0,15188.0,373.0,1776.0,3538.0,219.9
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014,493.1,57074.0,445.0,16051.0,10220.0,90.1
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014,197.2,80291.0,2152.0,9312.0,23406.0,138.6
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014,12.0,6929.0,135.0,497.0,1675.0,419.1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023,32.3,7993.0,229.0,1004.0,2126.0,163.9
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023,134.1,15420.0,190.0,3155.0,3154.0,188.0
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023,22.7,6042.0,410.0,384.0,1297.0,398.0
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023,8.6,4143.0,168.0,233.0,896.0,573.7


In [69]:
merged_df = merged_df.merge(ZonasRurais[['Concelho', 'Rural']], on='Concelho', how='left')
merged_df

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional,População Residente,Setor Primário,Setor Secundário,Setor Terciário,Indice de Envelhecimento,Rural
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014,2486.8,174592.0,492.0,10808.0,59539.0,142.4,0.000000
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014,34.0,15188.0,373.0,1776.0,3538.0,219.9,1.000000
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014,493.1,57074.0,445.0,16051.0,10220.0,90.1,0.800000
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014,197.2,80291.0,2152.0,9312.0,23406.0,138.6,0.923077
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014,12.0,6929.0,135.0,497.0,1675.0,419.1,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023,32.3,7993.0,229.0,1004.0,2126.0,163.9,1.000000
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023,134.1,15420.0,190.0,3155.0,3154.0,188.0,1.000000
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023,22.7,6042.0,410.0,384.0,1297.0,398.0,1.000000
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023,8.6,4143.0,168.0,233.0,896.0,573.7,1.000000


In [70]:
# Lista de todas as tabelas de clima
tabelas_clima = tabelas = [('T_MED', T_MED), ('T_MAX', T_MAX), ('T_MIN', T_MIN), ('DD_MED', DD_MED), ('DD_FFX', DD_FFX), ('FF_MED', FF_MED), ('FF_MAX', FF_MAX), ('PR_QTD', PR_QTD)]

# Iterar sobre todas as tabelas de clima
for tabela in tabelas_clima:
    # Obtendo o nome da tabela
    nome_tabela = tabela[0]
    TabeIPMA = tabela[1]
    # Iterar sobre as linhas de merged_df
    for index, row in merged_df.iterrows():

        ano = int(row['Ano'])
        mes = int(row['Mês'])
        dia = int(row['Dia'])
        concelho = row["Distrito"]

        
        valor = TabeIPMA[concelho][(TabeIPMA["Ano"] == ano) & (TabeIPMA["Mês"] == mes) & (TabeIPMA["Dia"] == dia)].iloc[0]
        
        # Atualizando a linha 'merged_df' com o valor obtido
        merged_df.at[index, f'{nome_tabela}'] = valor

# Exibindo o DataFrame merged_df atualizado
merged_df

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional,População Residente,...,Indice de Envelhecimento,Rural,T_MED,T_MAX,T_MIN,DD_MED,DD_FFX,FF_MED,FF_MAX,PR_QTD
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014,2486.8,174592.0,...,142.4,0.000000,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014,34.0,15188.0,...,219.9,1.000000,10.6,12.2,8.1,222.0,248.0,4.0,12.6,13.8
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014,493.1,57074.0,...,90.1,0.800000,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014,197.2,80291.0,...,138.6,0.923077,14.4,15.2,12.0,232.0,236.0,5.4,13.4,3.8
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014,12.0,6929.0,...,419.1,1.000000,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0,-990.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023,32.3,7993.0,...,163.9,1.000000,8.9,10.6,7.8,293.0,328.0,2.6,8.1,1.6
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023,134.1,15420.0,...,188.0,1.000000,-990.0,-990.0,-990.0,337.0,339.0,2.6,10.1,5.8
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023,22.7,6042.0,...,398.0,1.000000,6.9,9.9,4.9,264.0,208.0,2.9,12.7,0.8
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023,8.6,4143.0,...,573.7,1.000000,6.9,9.9,4.9,264.0,208.0,2.9,12.7,0.8


In [71]:
merged_df.replace(-990.0, None, inplace=True)
merged_df

,Data do incidente,Concelho,Distrito,Nível de Tensão,Número de Eventos,Dia,Mês,Ano,Densidade Populacional,População Residente,...,Indice de Envelhecimento,Rural,T_MED,T_MAX,T_MIN,DD_MED,DD_FFX,FF_MED,FF_MAX,PR_QTD
0,2014-01-01,Almada,Setúbal,Sem Eventos Excecionais,0,1,1,2014,2486.8,174592.0,...,142.4,0.000000,None,None,None,None,None,None,None,None
1,2014-01-01,Sertã,Castelo Branco,Sem Eventos Excecionais,0,1,1,2014,34.0,15188.0,...,219.9,1.000000,10.6,12.2,8.1,222.0,248.0,4.0,12.6,13.8
2,2014-01-01,Felgueiras,Porto,Sem Eventos Excecionais,0,1,1,2014,493.1,57074.0,...,90.1,0.800000,None,None,None,None,None,None,None,None
3,2014-01-01,Torres Vedras,Lisboa,Sem Eventos Excecionais,0,1,1,2014,197.2,80291.0,...,138.6,0.923077,14.4,15.2,12.0,232.0,236.0,5.4,13.4,3.8
4,2014-01-01,Nisa,Portalegre,Sem Eventos Excecionais,0,1,1,2014,12.0,6929.0,...,419.1,1.000000,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1008001,2023-12-31,Campo Maior,Portalegre,Sem Eventos Excecionais,0,31,12,2023,32.3,7993.0,...,163.9,1.000000,8.9,10.6,7.8,293.0,328.0,2.6,8.1,1.6
1008002,2023-12-31,Castelo De Paiva,Aveiro,Sem Eventos Excecionais,0,31,12,2023,134.1,15420.0,...,188.0,1.000000,None,None,None,337.0,339.0,2.6,10.1,5.8
1008003,2023-12-31,Vila Flor,Bragança,Sem Eventos Excecionais,0,31,12,2023,22.7,6042.0,...,398.0,1.000000,6.9,9.9,4.9,264.0,208.0,2.9,12.7,0.8
1008004,2023-12-31,Vimioso,Bragança,Sem Eventos Excecionais,0,31,12,2023,8.6,4143.0,...,573.7,1.000000,6.9,9.9,4.9,264.0,208.0,2.9,12.7,0.8


In [72]:
merged_df.to_parquet("AllDaysEventos.parquet", index=False)